In [48]:
import os
import math
import talib
import random
import pickle
import json
import pandas as pd
import numpy as np
import jqdatasdk as jq
import baostock as bs
import time

from multiprocessing import Pool
from functools import partial
from scipy import signal
from scipy import stats
from datetime import datetime, timedelta
from tqdm import tqdm

import matplotlib.pyplot as plt
from IPython.core.display import HTML, clear_output

%matplotlib inline
%config InlineBackend.figure_format='svg'
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows',    5000)

from featureutils import FeatureTool
from plotutils import PlotTool
from datagent import BaosAgent
from datagent import DataUtil
from datagent import JQDataUtil
from sfstrategy import SFStrategy

In [2]:
def get_freq_file_sto_ids(freq):
    file_sto_ids = []
    path_src = './data/' + str(freq) + '/'
    files = os.listdir(path_src)
    all_file_sto_id = []
    for file in files:
        if file.endswith('pkl'):
            file_sto_ids.append(file[:6])

    return file_sto_ids

def add_vol_subplot(num_subplots, idx_subplot, df):
    xaxis_idxes = df.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)
    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.fill_between(xaxis_idxes, 0, df.volume, facecolor='cyan')
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')

def add_ma_subplot(num_subplots, idx_subplot, df):
    xaxis_idxes = df.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)
    
    cond_ma_long  = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
    cond_ma_short = (df.CLOSE_MA_18d < df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d < df.CLOSE_MA_50d.shift(1))
    cond_ma_consolidation = ((df.CLOSE_MA_18d <= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d >= df.CLOSE_MA_50d.shift(1))) | \
                            ((df.CLOSE_MA_18d >= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d <= df.CLOSE_MA_50d.shift(1)))

    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.CLOSE_MA_18d, lw=.5, color='cyan')
    ax.plot(xaxis_idxes, df.CLOSE_MA_50d, lw=.5, color='lime')
    ax.plot(xaxis_idxes, df.close,        lw=.5, color='orange')
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_long,          facecolor='red',    alpha=0.1)
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_short,         facecolor='lime',   alpha=0.1)
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_consolidation, facecolor='orange', alpha=0.1)
    
    cond_bband_long_cross = (df.close > df.BBAND_upper) & (df.close.shift(1) < df.BBAND_upper.shift(1))
    cond_vol_rise         = (df.volume > df.volume.rolling(daily_count*5).mean() * 2)
    cond = cond_bband_long_cross & cond_vol_rise & cond_ma_long
    for tgt_idx in df[cond].index:
        cur_p = df.iloc[tgt_idx].close
        max_p = np.max(df.close.iloc[tgt_idx + 1:tgt_idx+daily_count*5])
        min_p = np.min(df.close.iloc[tgt_idx + 1:tgt_idx+daily_count*5])
        max_cr = (max_p - cur_p) / cur_p
        min_cr = (min_p - cur_p) / cur_p
        print('{:>6} {:>6.2%} {:>6.2%}'.format(tgt_idx, max_cr, min_cr))
#     for tgt_idx in df[cond].index:
        ax.vlines(xaxis_idxes[tgt_idx], np.min(df.close), np.max(df.close), lw=1, color='w')
        
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')
    
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.fill_between(xaxis_idxes, 0, df.volume/100, facecolor='cyan')
    plt.sca(ax2)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
    plt.yticks(fontsize=8, color='w')

def add_bband_subplot(num_subplots, idx_subplot, df):
    xaxis_idxes = df.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)

    cond_bband_long  = (df.close > df.BBAND_upper)
    cond_bband_short = (df.close < df.BBAND_lower)

    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.BBAND_upper, lw=.3, color='red')
    ax.plot(xaxis_idxes, df.BBAND_lower, lw=.3, color='lime')
    ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_bband_long,  facecolor='red',  alpha=0.1)
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_bband_short, facecolor='lime', alpha=0.1)
    
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')
    

def compute_longshort_index(record):
    long = 0
    if record.k_qut > record.d_qut and record.k_qut > 20 or record.k_qut > 80:
        long += 4
    if record.k_mon > record.d_mon and record.d_mon > 20 or record.d_mon > 80:
        long += 3
    if record.k_wek > record.d_wek and record.d_wek > 20 or record.d_wek > 80:
        long += 2
    if record.k_day > record.d_day and record.d_day > 20 or record.d_day > 80:
        long += 1

    short = 0
    if record.k_qut < record.d_qut and record.k_qut < 80 or record.k_qut < 20:
        short += 4
    if record.k_mon < record.d_mon and record.d_mon < 80 or record.d_mon < 20:
        short += 3
    if record.k_wek < record.d_wek and record.d_wek < 80 or record.d_wek < 20:
        short += 2
    if record.k_day < record.d_day and record.d_day < 80 or record.d_day < 20:
        short += 1

    return long, short

def compute_k_above60_sum(x):
    return np.sum(x[x > 60])

def compute_k_below40_sum(x):
    return np.sum(x[x < 40])

In [3]:
def add_kd_subplot(num_subplots, idx_subplot, k, d, high_saturation, low_saturation, cond_long, cond_short, closes, cond_all_long):
    xaxis_idxes = k.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)    

    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, k, lw=.3, color='cyan')
    ax.plot(xaxis_idxes, d, lw=.3, color='orange')
    ax.plot(xaxis_idxes[high_saturation], k[high_saturation], '.', color='cyan', alpha=.5)
    ax.plot(xaxis_idxes[low_saturation],  k[low_saturation],  '.', color='cyan', alpha=.5)
    
    ax.fill_between(xaxis_idxes, 55, k, where=k>55, facecolor='orangered', alpha=0.3, interpolate=True)
    ax.fill_between(xaxis_idxes, k, 45, where=k<45, facecolor='lime',      alpha=0.3, interpolate=True)
    ax.hlines(20, 0, max(xaxis_idxes), lw=.1, color='w')
    ax.hlines(40, 0, max(xaxis_idxes), lw=.1, color='w')
    ax.hlines(50, 0, max(xaxis_idxes), lw=.1, color='w')
    ax.hlines(60, 0, max(xaxis_idxes), lw=.1, color='w')
    ax.hlines(80, 0, max(xaxis_idxes), lw=.1, color='w')
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, closes, lw=.5, color='orange')
    ax2.fill_between(xaxis_idxes, min(closes), max(closes), where=cond_long,     facecolor='orangered', alpha=0.1)
    ax2.fill_between(xaxis_idxes, min(closes), max(closes), where=cond_all_long, facecolor='orangered', alpha=0.3)
    ax2.fill_between(xaxis_idxes, min(closes), max(closes), where=cond_short,    facecolor='lime',      alpha=0.1)
    tgt_idxes = []
    cond_gcross = (k > d) & (k.shift(1) < d.shift(1))
    cond_dcross = (k < d) & (k.shift(1) > d.shift(1))
    gcross_idxes = df[cond_gcross].index
    dcross_idxes = df[cond_dcross].index
    if len(gcross_idxes) > 1:
        for idx in range(len(gcross_idxes) - 1):
            first_gcross_idx = gcross_idxes[idx]
            second_gcross_idx = gcross_idxes[idx+1]
            
            first_gcross_k, second_gcross_k = k.iloc[first_gcross_idx], k.iloc[second_gcross_idx]
            first_gcross_close, second_gcross_close = df.close.iloc[first_gcross_idx], df.close.iloc[second_gcross_idx]
            if first_gcross_k < second_gcross_k and first_gcross_close > second_gcross_close:
                tgt_idxes.append(second_gcross_idx)
        print(idx_subplot, tgt_idxes)
        for tgt_idx in tgt_idxes:
            ax2.vlines(xaxis_idxes[tgt_idx], np.min(df.close), np.max(df.close), lw=1, color='w')
    
    plt.sca(ax2)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')

def add_kds_indicators(df):
    factor_4w = daily_count * 20
    factor_3w = daily_count * 15
    factor_2w = daily_count * 10
    factor_5d = daily_count * 5
    factor_4d = daily_count * 4
    factor_3d = daily_count * 3
    factor_2d = daily_count * 2
    factor_1d = daily_count * 1
    factor_1h = 4
    factor_1t = 1

    df['k_4w'], df['d_4w'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_4w, slowk_period=3*factor_4w, slowd_period=3*factor_4w)
    df['k_3w'], df['d_3w'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_3w, slowk_period=3*factor_3w, slowd_period=3*factor_3w)
    df['k_2w'], df['d_2w'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_2w, slowk_period=3*factor_2w, slowd_period=3*factor_2w)
    df['k_5d'], df['d_5d'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_5d, slowk_period=3*factor_5d, slowd_period=3*factor_5d)
    df['k_4d'], df['d_4d'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_4d, slowk_period=3*factor_4d, slowd_period=3*factor_4d)
    df['k_3d'], df['d_3d'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_3d, slowk_period=3*factor_3d, slowd_period=3*factor_3d)
    df['k_2d'], df['d_2d'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_2d, slowk_period=3*factor_2d, slowd_period=3*factor_2d)
    df['k_1d'], df['d_1d'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_1d, slowk_period=3*factor_1d, slowd_period=3*factor_1d)
    df['k_1h'], df['d_1h'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_1h, slowk_period=3*factor_1h, slowd_period=3*factor_1h)
    df['k_1t'], df['d_1t'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_1t, slowk_period=3*factor_1t, slowd_period=3*factor_1t)
    
    return df

def add_volrsi_indicators(df):
    factor_8t = daily_count * 20
    factor_7t = daily_count * 15
    factor_6t = daily_count * 10
    factor_5t = daily_count * 5
    factor_4t = daily_count * 4
    factor_3t = daily_count * 3
    factor_2t = daily_count * 2
    factor_1t = daily_count * 1

    df['vol_rsi_8t'] = talib.RSI(df.volume, timeperiod=factor_8t)
    df['vol_rsi_7t'] = talib.RSI(df.volume, timeperiod=factor_7t)
    df['vol_rsi_6t'] = talib.RSI(df.volume, timeperiod=factor_6t)
    df['vol_rsi_5t'] = talib.RSI(df.volume, timeperiod=factor_5t)
    df['vol_rsi_4t'] = talib.RSI(df.volume, timeperiod=factor_4t)
    df['vol_rsi_3t'] = talib.RSI(df.volume, timeperiod=factor_3t)
    df['vol_rsi_2t'] = talib.RSI(df.volume, timeperiod=factor_2t)
    df['vol_rsi_1t'] = talib.RSI(df.volume, timeperiod=factor_1t)
    
    return df

def add_mfi_indicators(df):
    factor_8t = daily_count * 20
    factor_7t = daily_count * 15
    factor_6t = daily_count * 10
    factor_5t = daily_count * 5
    factor_4t = daily_count * 4
    factor_3t = daily_count * 3
    factor_2t = daily_count * 2
    factor_1t = daily_count * 1
    factor_1h = 4

    df['mfi_8t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_8t)
    df['mfi_7t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_7t)
    df['mfi_6t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_6t)
    df['mfi_5t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_5t)
    df['mfi_4t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_4t)
    df['mfi_3t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_3t)
    df['mfi_2t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_2t)
    df['mfi_1t'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_1t)
    df['mfi_1h'] = talib.MFI(df.high, df.low, df.close, np.log(df.volume + 1), timeperiod=factor_1h)
    
#     df['mfi_1t_1week_mean']   = df.mfi_1t.rolling(daily_count * 5).mean()
#     df['mfi_1t_1week_median'] = df.mfi_1t.rolling(daily_count * 5).median()
#     df['mfi_1t_1week_std']    = df.mfi_1t.rolling(daily_count * 5).std()
    
    return df

def add_volma_indicators(df):
    factor_4w = daily_count * 20
    factor_3w = daily_count * 15
    factor_2w = daily_count * 10
    factor_5d = daily_count * 5
    factor_4d = daily_count * 4
    factor_3d = daily_count * 3
    factor_2d = daily_count * 2
    factor_1d = daily_count * 1
    factor_1h = 4
    
    df.loc[df.volume==0, 'volume'] = 5000
    df['vol_log_ma_4w'] = talib.MA(np.log(df.volume), timeperiod=factor_4w)
    df['vol_log_ma_3w'] = talib.MA(np.log(df.volume), timeperiod=factor_3w)
    df['vol_log_ma_2w'] = talib.MA(np.log(df.volume), timeperiod=factor_2w)
    df['vol_log_ma_5d'] = talib.MA(np.log(df.volume), timeperiod=factor_5d)
    df['vol_log_ma_4d'] = talib.MA(np.log(df.volume), timeperiod=factor_4d)
    df['vol_log_ma_3d'] = talib.MA(np.log(df.volume), timeperiod=factor_3d)
    df['vol_log_ma_2d'] = talib.MA(np.log(df.volume), timeperiod=factor_2d)
    df['vol_log_ma_1d'] = talib.MA(np.log(df.volume), timeperiod=factor_1d)
    df['vol_log_ma_1h'] = talib.MA(np.log(df.volume), timeperiod=factor_1h)

    df['vol_log_ma_avg'] = (df.vol_log_ma_1h + df.vol_log_ma_1d*4 + df.vol_log_ma_2d*8 + df.vol_log_ma_3d*12 + df.vol_log_ma_4d*16 + df.vol_log_ma_5d*20 + df.vol_log_ma_2w*40 + df.vol_log_ma_3w*60 + df.vol_log_ma_4w*80) / (1+4+8+12+16+20+40+60+80)
    
    return df

def add_closema_indicators(df):
    factor_4w = daily_count * 20
    factor_3w = daily_count * 15
    factor_2w = daily_count * 10
    factor_5d = daily_count * 5
    factor_4d = daily_count * 4
    factor_3d = daily_count * 3
    factor_2d = daily_count * 2
    factor_1d = daily_count * 1
    factor_1h = 4

    df['closema_4w'] = talib.MA(df.close, timeperiod=factor_4w)
    df['closema_3w'] = talib.MA(df.close, timeperiod=factor_3w)
    df['closema_2w'] = talib.MA(df.close, timeperiod=factor_2w)
    df['closema_5d'] = talib.MA(df.close, timeperiod=factor_5d)
    df['closema_4d'] = talib.MA(df.close, timeperiod=factor_4d)
    df['closema_3d'] = talib.MA(df.close, timeperiod=factor_3d)
    df['closema_2d'] = talib.MA(df.close, timeperiod=factor_2d)
    df['closema_1d'] = talib.MA(df.close, timeperiod=factor_1d)
    df['closema_1h'] = talib.MA(df.close, timeperiod=factor_1h)
    
    df['closema_avg'] = (df.closema_1h + df.closema_1d*4 + df.closema_2d*8 + df.closema_3d*12 + df.closema_4d*16 + df.closema_5d*20 + df.closema_2w*40 + df.closema_3w*60 + df.closema_4w*80) / (1+4+8+12+16+20+40+60+80)
    
    return df

def add_rsi_indicators(df):
    factor_8t = daily_count * 20
    factor_7t = daily_count * 15
    factor_6t = daily_count * 10
    factor_5t = daily_count * 5
    factor_4t = daily_count * 4
    factor_3t = daily_count * 3
    factor_2t = daily_count * 2
    factor_1t = daily_count * 1
    factor_1h = 4

    df['rsi_8t'] = talib.RSI(df.close, timeperiod=factor_8t)
    df['rsi_7t'] = talib.RSI(df.close, timeperiod=factor_7t)
    df['rsi_6t'] = talib.RSI(df.close, timeperiod=factor_6t)
    df['rsi_5t'] = talib.RSI(df.close, timeperiod=factor_5t)
    df['rsi_4t'] = talib.RSI(df.close, timeperiod=factor_4t)
    df['rsi_3t'] = talib.RSI(df.close, timeperiod=factor_3t)
    df['rsi_2t'] = talib.RSI(df.close, timeperiod=factor_2t)
    df['rsi_1t'] = talib.RSI(df.close, timeperiod=factor_1t)
    df['rsi_1h'] = talib.RSI(df.close, timeperiod=factor_1h)
    
    return df

def add_macd_indicators(df):
    factor_4w = daily_count * 20
    factor_3w = daily_count * 15
    factor_2w = daily_count * 10
    factor_5d = daily_count * 5
    factor_4d = daily_count * 4
    factor_3d = daily_count * 3
    factor_2d = daily_count * 2
    factor_1d = daily_count * 1
    factor_1h = 4
    factor_1t = 1
    
    df['macd_4w'], df['macdsignal_4w'], df['macdhist_4w'] = talib.MACD(df.close, fastperiod=12*factor_4w, slowperiod=26*factor_4w, signalperiod=9*factor_4w)
    df['macd_3w'], df['macdsignal_3w'], df['macdhist_3w'] = talib.MACD(df.close, fastperiod=12*factor_3w, slowperiod=26*factor_3w, signalperiod=9*factor_3w)
    df['macd_2w'], df['macdsignal_2w'], df['macdhist_2w'] = talib.MACD(df.close, fastperiod=12*factor_2w, slowperiod=26*factor_2w, signalperiod=9*factor_2w)
    df['macd_1w'], df['macdsignal_1w'], df['macdhist_1w'] = talib.MACD(df.close, fastperiod=12*factor_5d, slowperiod=26*factor_5d, signalperiod=9*factor_5d)
    df['macd_4d'], df['macdsignal_4d'], df['macdhist_4d'] = talib.MACD(df.close, fastperiod=12*factor_4d, slowperiod=26*factor_4d, signalperiod=9*factor_4d)
    df['macd_3d'], df['macdsignal_3d'], df['macdhist_3d'] = talib.MACD(df.close, fastperiod=12*factor_3d, slowperiod=26*factor_3d, signalperiod=9*factor_3d)
    df['macd_2d'], df['macdsignal_2d'], df['macdhist_2d'] = talib.MACD(df.close, fastperiod=12*factor_2d, slowperiod=26*factor_2d, signalperiod=9*factor_2d)
    df['macd_1d'], df['macdsignal_1d'], df['macdhist_1d'] = talib.MACD(df.close, fastperiod=12*factor_1d, slowperiod=26*factor_1d, signalperiod=9*factor_1d)
    df['macd_1h'], df['macdsignal_1h'], df['macdhist_1h'] = talib.MACD(df.close, fastperiod=12*factor_1h, slowperiod=26*factor_1h, signalperiod=9*factor_1h)
    df['macd_1t'], df['macdsignal_1t'], df['macdhist_1t'] = talib.MACD(df.close, fastperiod=12*factor_1t, slowperiod=26*factor_1t, signalperiod=9*factor_1t)
    
    return df

def compute_cond_long(k, d):
    cond_long  = (k > d) & (k > 20) | (k > 80)
    return cond_long

def compute_cond_short(k, d):
    cond_short = (k < d) & (k < 80) | (k < 20)
    return cond_short

def compute_high_saturation(k):
    cond_high_saturation = (k > 80).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
    return cond_high_saturation

def compute_low_saturation(k):
    cond_low_saturation  = (k < 20).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
    return cond_low_saturation

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

### Find Targets

In [50]:
delta = 15
daily_count = int(240/delta)
factor_qut = daily_count * 64
factor_mon = daily_count * 16
factor_wek = daily_count * 4
factor_day = daily_count * 1
factor_60m = 4
factor_30m = 2
factor_15m = 1


def find_all_crabove20in1week(sub_sto_ids):
    match_infos = []
    for sto_id in tqdm(sub_sto_ids):
        df = BaosAgent.read_freq_df(delta, sto_id)
        df = df.dropna()
        df = df.reset_index(drop=True)
        df['close_max_in1week'] = df.close.rolling(daily_count * 5).max().shift(-daily_count * 5)
        df['close_min_in1week'] = df.close.rolling(daily_count * 5).min().shift(-daily_count * 5)
        df['cr_max_in1week']    = (df.close_max_in1week - df.close) / df.close
        df['cr_min_in1week']    = (df.close_min_in1week - df.close) / df.close
        cond = (df.cr_max_in1week > .2) & (df.cr_min_in1week > 0)
        if len(cond) > 0:
            for idx in df[cond].index:
                match_infos.append([sto_id, df.iloc[idx].time])
    return match_infos

def find_all_crabove20in1week_callback(rlts):
    all_match_infos = []
    for rlt in rlts:
        for match_info in rlt:
            all_match_infos.append(match_info)
    print(len(all_match_infos))
    return all_match_infos
    

def find_targets(sub_sto_ids):
    match_sto_ids = []
    for sto_id in tqdm(sub_sto_ids):
        df = BaosAgent.read_freq_df(delta, sto_id)
        df = add_macd_indicators(df)
        df = df.dropna()
        df = df.reset_index(drop=True)
        cond_macd_long = (df.macd_1t > df.macd_1t.shift(1)) & (df.macd_2t > df.macd_2t.shift(1)) & (df.macd_3t > df.macd_3t.shift(1)) & (df.macd_4t > df.macd_4t.shift(1)) & (df.macd_5t > df.macd_5t.shift(1)) & (df.macd_1h > df.macd_1h.shift(1))
        if cond_macd_long.iloc[-1]:
            match_sto_ids.append(sto_id)
    return match_sto_ids

def find_targets_callback(rlts):
    match_sto_infos = []
    for rlt in rlts:
        match_sto_infos.extend(rlt)
    print(len(match_sto_infos))
    return match_sto_infos

def find_macd_allabove0(sub_sto_ids, target_idx):
    match_infos = []
    for sto_id in tqdm(sub_sto_ids):
        df = BaosAgent.read_freq_df(delta, sto_id)
        df = add_macd_indicators(df)
        df = df.dropna()
        df = df.reset_index(drop=True)
        if len(df) < abs(target_idx):
            continue
        
        cond_macd_allabove0 = (df.macd_1h > 0) & (df.macd_1t > 0) & (df.macd_2t > 0) & (df.macd_3t > 0) & (df.macd_4t > 0) & (df.macd_5t > 0)
        if len(cond_macd_allabove0) > 0 and not cond_macd_allabove0.iloc[target_idx-1] and cond_macd_allabove0.iloc[target_idx]:
            print(sto_id, df.iloc[target_idx].time)
            match_infos.append([sto_id, df.iloc[target_idx].time])
    return match_infos
def find_macd_allabove0_callback(rlts):
    all_match_infos = []
    for rlt in rlts:
        all_match_infos.extend(rlt)
    print(len(all_match_infos))
    return all_match_infos


def find_last1w_vols(sub_sto_ids, target_idx):
    match_infos = []
    for sto_id in tqdm(sub_sto_ids):
        df = BaosAgent.read_freq_df(delta, sto_id)
        if len(df) < daily_count * 5 * 40:
            continue
        
        cur_idx = target_idx
        vol_increase_rate = df.iloc[cur_idx - daily_count*5:cur_idx].volume.sum() / df.iloc[cur_idx - daily_count*5 * 25:cur_idx - daily_count*5].volume.sum()
        if vol_increase_rate > .5:
            print(sto_id, df.iloc[cur_idx].time, vol_increase_rate)
            match_infos.append([sto_id, df.iloc[cur_idx].time])
    return match_infos
def find_last1w_vols_callback(rlts):
    all_match_infos = []
    for rlt in rlts:
        all_match_infos.extend(rlt)
    print(len(all_match_infos))
    return all_match_infos

def find_macd_dwlong(sub_sto_ids, target_idx):
    jq.auth('15808061188', 'allan2jq')
    match_infos = []
    minimum_size = 11198 + daily_count * 12 * 5
    factor_4w = daily_count * 20
    factor_3w = daily_count * 15
    factor_2w = daily_count * 10
    factor_5d = daily_count * 5
    factor_4d = daily_count * 4
    factor_3d = daily_count * 3
    factor_2d = daily_count * 2
    factor_1d = daily_count * 1
    for tmp_sto_id in tqdm(sub_sto_ids):
        tmp_df = BaosAgent.read_freq_df(delta, tmp_sto_id)
        if len(tmp_df) < minimum_size or tmp_df.iloc[target_idx].close > 20:
            continue
        
        # print('get realtime data from JQData ...')
#         tmp_df = DataUtil.add_rt_data(tmp_sto_id, tmp_df, delta, JQDataUtil.get_end_ts(delta))
        tmp_df = DataUtil.add_rt_data(tmp_sto_id, tmp_df, delta, '2020-02-24 10:45:00')
        # print('get realtime data from JQData accomplished.')

        tmp_df = tmp_df[-minimum_size:].copy()
        tmp_df = tmp_df.reset_index(drop=True)

#         t1 = datetime.now()
        tmp_df = add_macd_indicators(tmp_df)
#         tmp_df = add_kds_indicators(tmp_df)
        tmp_df['k_1t'], tmp_df['d_1t'] = talib.STOCH(tmp_df.high, tmp_df.low, tmp_df.close, fastk_period=9, slowk_period=3, slowd_period=3)
        tmp_df = tmp_df.dropna()
        tmp_df = tmp_df.reset_index(drop=True)
#         t2 = datetime.now()
        cond_macd_dlong = (tmp_df.macdhist_1d > tmp_df.macdhist_1d.shift(1)) & (tmp_df.macdhist_2d > tmp_df.macdhist_2d.shift(1)) & (tmp_df.macdhist_3d > tmp_df.macdhist_3d.shift(1)) & (tmp_df.macdhist_4d > tmp_df.macdhist_4d.shift(1))
        cond_macd_wlong = (tmp_df.macdhist_1w > tmp_df.macdhist_1w.shift(1)) & (tmp_df.macdhist_2w > tmp_df.macdhist_2w.shift(1)) & (tmp_df.macdhist_3w > tmp_df.macdhist_3w.shift(1)) & (tmp_df.macdhist_4w > tmp_df.macdhist_4w.shift(1))
        cond_macd_1t1h_increase = (tmp_df.macdhist_1t > tmp_df.macdhist_1t.shift(1)) & (tmp_df.macdhist_1h > tmp_df.macdhist_1h.shift(1))
        cond = cond_macd_dlong & cond_macd_wlong & cond_macd_1t1h_increase
        
        cond_k1t_minimum = (tmp_df.k_1t > tmp_df.k_1t.shift(1)) & (tmp_df.k_1t.shift(1) < tmp_df.k_1t.shift(2)) & cond
        if len(tmp_df[cond_k1t_minimum]) < 2:
            continue
        cur_idx = tmp_df.index[target_idx]
        minimum_idxes = tmp_df[cond_k1t_minimum].index.values
#         t3 = datetime.now()
        if cur_idx in minimum_idxes:
            pre_minimum_idx = minimum_idxes[minimum_idxes <= cur_idx][-2]
            if cond.iloc[cur_idx] and tmp_df.k_1t.iloc[cur_idx - 1] > tmp_df.k_1t.iloc[pre_minimum_idx - 1]:# and tmp_df.close.iloc[cur_idx - 1] < tmp_df.close.iloc[pre_minimum_idx - 1]:
#                 t4 = datetime.now()
                print(tmp_sto_id, tmp_df.iloc[cur_idx].time, tmp_df.k_1t.iloc[cur_idx-2:].values, tmp_df.iloc[-1].time)#, t2-t1, t3-t2, t4-t3, t4-t1
                match_infos.append([tmp_sto_id, tmp_df.iloc[target_idx].time])
        
#     jq.logout()
    return match_infos
def find_macd_dwlong_callback(rlts):
    all_match_infos = []
    for rlt in rlts:
        all_match_infos.extend(rlt)
    print(len(all_match_infos))
    return all_match_infos

In [55]:

sto_ids = get_freq_file_sto_ids(delta)
# sto_ids = backup_sto_ids
sto_ids = sto_ids[:360]
NUM_PROCESS = 3
size = math.ceil(len(sto_ids) / NUM_PROCESS)
sub_seqs = []
for proc_idx in range(NUM_PROCESS):
    start_idx = size * proc_idx
    stop_idx = size * (proc_idx + 1)
    sub_seq = sto_ids[start_idx:stop_idx]
    sub_seqs.append(sub_seq)

obs_idx = -1
with Pool(NUM_PROCESS) as pool:
    rlts = pool.map(partial(find_macd_dwlong, target_idx=obs_idx), sub_seqs)
    all_match_infos = find_macd_dwlong_callback(rlts)




auth success 


  0%|          | 0/120 [00:00<?, ?it/s]

auth success 


  0%|          | 0/120 [00:00<?, ?it/s]

auth success 


...

600478 202002241045 [82.05128205 65.38461538 71.79487179] 202002241045


 94%|█████████▍| 113/120 [00:14<00:00,  7.57it/s]

600527 202002241045 [79.62962963 77.77777778 85.18518519] 202002241045


100%|██████████| 120/120 [00:16<00:00,  7.15it/s]


5


In [16]:
JQDataUtil.get_end_ts(delta)

'2020-02-24 10:30:00'

In [38]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

{'total': 10000000, 'spare': 9513648}

In [42]:
find_macd_dwlong(['300309'], -1)

100%|██████████| 1/1 [00:00<00:00,  5.64it/s]


[]

In [ ]:
all_df = None
for sto_id in tqdm(all_crabove20in1week_sto_ids):
    match_ts = all_crabove20in1week_df[all_crabove20in1week_df.sto_id == sto_id].ts.values.tolist()
    df = BaosAgent.read_freq_df(delta, sto_id)
    df = add_volma_indicators(df)
    df = add_closema_indicators(df)
    df = add_rsi_indicators(df)
    df = add_mfi_indicators(df)
    df = add_macd_indicators(df)
    df = add_kds_indicators(df)
    df = df.dropna()
    df = df.reset_index(drop=True)
    df = df[df.time.isin(match_ts)]
    df = df.reset_index(drop=True)
    if len(df) > 0:
        if all_df is not None:
            all_df = pd.concat([all_df, df], ignore_index=True)
        else:
            all_df = df

In [ ]:
all_df = pd.read_csv('all.csv', dtype={'time':str, 'code':str})
all_match_sto_ids = all_df.code.unique().tolist()

In [62]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

###### delta = 15
daily_count = int(240/delta)

### Prepare sto base info.
# sto_ids = get_freq_file_sto_ids(delta)
# sto_id = random.sample(all_match_sto_ids, 1)[0]
sto_id = random.sample(all_match_infos, 1)[0][0]
for match_info in all_match_infos:
    sto_id = match_info[0]
    # sto_id = '600712'
    print(sto_id)
    ### Prepare sto data.
    tmp_df = BaosAgent.read_freq_df(delta, sto_id)
    ### get realtime data from JQData ...')
    tmp_df = DataUtil.add_rt_data(sto_id, tmp_df, delta, '2020-02-24 10:45:00')
    ### print('get realtime data from JQData accomplished.')
    l1 = len(tmp_df)
    ### Add realtime data.
    # print('get realtime data from JQData ...')
    # df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
    # print('get realtime data from JQData accomplished.')

    ### Add TA features.
    tmp_df = add_volma_indicators(tmp_df)
    tmp_df = add_closema_indicators(tmp_df)
    tmp_df = add_rsi_indicators(tmp_df)
    tmp_df = add_mfi_indicators(tmp_df)
    tmp_df = add_macd_indicators(tmp_df)
    tmp_df = add_kds_indicators(tmp_df)
    tmp_df['corr_closema_volma'] = tmp_df['close'].rolling(4).corr(tmp_df['volume'])

    ### Clean up data.
    tmp_df = tmp_df.dropna()
    l2 = len(tmp_df)
    tmp_df = tmp_df.iloc[-daily_count * 5 * 4:]

    tmp_df = tmp_df.reset_index(drop=True)
    print(sto_id, len(tmp_df), l1 - l2)
    ### Plot figure.
    if len(tmp_df) > 0:
    #     target_ts = all_df[all_df.code == sto_id].time.tolist()
    #     target_idxes = df[df.time.isin(target_ts)].index.tolist()
    #     target_idxes = []
    #     print(target_idxes)
        print(tmp_df.time.iloc[0], tmp_df.time.iloc[-1])
        idx = -daily_count*5*6 - 8
        print(sto_id, tmp_df.iloc[obs_idx-2:].k_1t.values)
#         plot_figure(tmp_df)

    ### 量升价升 royalblue
    ### 量升价跌 cyan
    ### 量跌价升 orangered
    ### 量跌价跌 red

    ### 

jq.logout()

600478
600478 320 11198
202001171100 202002241045
600478 [82.05128205 65.38461538 71.79487179]
002591
002591 320 11199
202001171100 202002241045
002591 [75.49019608 73.40425532 77.82741993]
600718
600718 320 11198
202001171100 202002241045
600718 [86.40350877 80.26315789 85.13030148]
300074
300074 320 11199
202001171100 202002241045
300074 [72.77614858 68.18181818 71.34680135]
600527
600527 320 11198
202001171100 202002241045
600527 [79.62962963 77.77777778 85.18518519]
已退出


In [ ]:
slopes = df.macd_1t.rolling(4).apply(compute_slopes, raw=True)
corrs  = df.macdhist_1t.rolling(4).corr(df.close)

fig = plt.figure(figsize=(.5 * len(df)/daily_count, 1 * 2))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)

ax = plt.subplot(1,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, slopes, lw=.5, color='orange')
ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.3)
custom_axticks(ax, xaxis_idxes, x_axis_slice)
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, corrs,  lw=.5, color='cyan')
ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.3)
custom_axticks(ax2, xaxis_idxes, x_axis_slice)
align_yaxis(ax, ax2, 0)

In [7]:
from scipy import stats

def compute_slopes(x):
    slope, intercept, r_value, p_value, std_err = stats.linregress(range(len(x)), x)
    return slope

def align_yaxis(ax1, ax2, base):
    ax1_ymin, ax1_ymax = ax1.get_ylim()
    ax2_ymin, ax2_ymax = ax2.get_ylim()
    
    new_ax2_ymax = (base-ax2_ymin) / ((base-ax1_ymin) / (ax1_ymax - ax1_ymin)) +  ax2_ymin
    new_ax1_ymax = (base-ax1_ymin) / ((base-ax2_ymin) / (ax2_ymax - ax2_ymin)) +  ax1_ymin
    if new_ax2_ymax >= ax2_ymax:
        ax2.set_ylim(ax2_ymin, new_ax2_ymax)
    else:
        ax1.set_ylim(ax1_ymin, new_ax1_ymax)

plt.rcParams['figure.dpi'] = 512

def custom_axticks(ax, xaxis_idxes, x_axis_slice):
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')

def plot_figure(df):
#     obs_idx = -20

    subplot_num = 7
    subplot_idx = 0
    fig = plt.figure(figsize=(1 * len(df)/daily_count, subplot_num * 2))
    fig.patch.set_facecolor('#2A2A2A')
    xaxis_idxes = df.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)

    
    
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.fill_between(xaxis_idxes, 0, df.volume/10000, facecolor='cyan')
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    
    ####################################
    ### 量价相关性
    ####################################
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
    cond_tgt = (df.closema_avg > df.closema_avg.shift(1)) & (df.closema_avg.shift(1) < df.closema_avg.shift(2))
#     for target_idx in df[cond_tgt].index:
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.fill_between(xaxis_idxes, 0, df.corr_closema_volma, facecolor='royalblue',   alpha=.2)
#     ### 量升价升
#     ax2.fill_between(xaxis_idxes, 0, df.corr_closema_volma, where=(df.closema_avg > df.closema_avg.shift(1)) & (df.vol_log_ma_avg > df.vol_log_ma_avg.shift(1)), facecolor='royalblue',   alpha=.2)
#     ### 量升价跌
#     ax2.fill_between(xaxis_idxes, df.corr_closema_volma, 0, where=(df.closema_avg < df.closema_avg.shift(1)) & (df.vol_log_ma_avg > df.vol_log_ma_avg.shift(1)), facecolor='cyan',   alpha=.2)
#     ### 量跌价升
#     ax2.fill_between(xaxis_idxes, df.corr_closema_volma, 0, where=(df.closema_avg > df.closema_avg.shift(1)) & (df.vol_log_ma_avg < df.vol_log_ma_avg.shift(1)), facecolor='orange', alpha=.2)
#     ### 量跌价跌
#     ax2.fill_between(xaxis_idxes, 0, df.corr_closema_volma, where=(df.closema_avg < df.closema_avg.shift(1)) & (df.vol_log_ma_avg < df.vol_log_ma_avg.shift(1)), facecolor='orangered',    alpha=.2)
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    
    ####################################
    ### 量价移动平均线
    ####################################
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    cond_closema_duotou = (df.closema_1d > df.closema_2d) & (df.closema_2d > df.closema_3d) & (df.closema_3d > df.closema_4d) & (df.closema_4d > df.closema_5d)# & (df.closema_5d > df.closema_2w) & (df.closema_2w > df.closema_3w) & (df.closema_3w > df.closema_4w)
    cond_volma_duotou = (df.vol_log_ma_1d > df.vol_log_ma_2d) & (df.vol_log_ma_2d > df.vol_log_ma_3d) & (df.vol_log_ma_3d > df.vol_log_ma_4d) & (df.vol_log_ma_4d > df.vol_log_ma_5d)# & (df.vol_log_ma_5d > df.vol_log_ma_2w) & (df.vol_log_ma_2w > df.vol_log_ma_3w) & (df.vol_log_ma_3w > df.vol_log_ma_4w)
    cond_closema_increase = df.closema_avg > df.closema_avg.shift(1)
    cond_volma_increase = (df.vol_log_ma_avg > df.vol_log_ma_avg.shift(1))
    
    PlotTool.custom_ax(ax)
#     ax.plot(xaxis_idxes, df.close,      lw=.5, color='orange')
    ax.plot(xaxis_idxes, df.closema_4w, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_3w, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_2w, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_5d, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_4d, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_3d, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_2d, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_1d, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_1h, lw=.1, color='orange')
    ax.plot(xaxis_idxes, df.closema_avg, lw=.5, color='orange')
#     for target_idx in target_idxes:
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    min_y = min(min(df.closema_4w), min(df.closema_3w), min(df.closema_2w), min(df.closema_5d), min(df.closema_4d), min(df.closema_3d), min(df.closema_2d), min(df.closema_1d), min(df.closema_1h))
    max_y = max(min(df.closema_4w), min(df.closema_3w), min(df.closema_2w), max(df.closema_5d), max(df.closema_4d), max(df.closema_3d), max(df.closema_2d), max(df.closema_1d), max(df.closema_1h))

#     ax.fill_between(xaxis_idxes, min_y, max_y, where=cond_closema_duotou & cond_volma_duotou & df.corr_closema_volma > 0, facecolor='orange',  alpha=0.2)
    ax.fill_between(xaxis_idxes, min_y, max_y, where=cond_closema_increase, facecolor='orange',  alpha=0.1)
    
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, df.vol_log_ma_4w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_3w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_2w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_5d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_4d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_3d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_2d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_1d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_1h, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.vol_log_ma_avg, lw=.5, color='cyan')
    min_y = min(min(df.vol_log_ma_4w), min(df.vol_log_ma_3w), min(df.vol_log_ma_2w), min(df.vol_log_ma_5d), min(df.vol_log_ma_4d), min(df.vol_log_ma_3d), min(df.vol_log_ma_2d), min(df.vol_log_ma_1d), min(df.vol_log_ma_1h))
    max_y = max(min(df.vol_log_ma_4w), min(df.vol_log_ma_3w), min(df.vol_log_ma_2w), max(df.vol_log_ma_5d), max(df.vol_log_ma_4d), max(df.vol_log_ma_3d), max(df.vol_log_ma_2d), max(df.vol_log_ma_1d), max(df.vol_log_ma_1h))
    
#     ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_closema_duotou & cond_volma_duotou & df.corr_closema_volma > 0, facecolor='cyan',  alpha=0.2)
    ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_volma_increase, facecolor='cyan',  alpha=0.1)
    
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ####################################
    ### macd_1t斜率，macd_1t与价格相关性
    ####################################
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    slopes = df.macdhist_1t.rolling(4).apply(compute_slopes, raw=True)
    corrs  = df.macdhist_1t.rolling(4).corr(df.close)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, slopes, lw=.5, color='orange')
    ax.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.3)
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, corrs,  lw=.5, color='cyan')
    ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.3)
    ax2.vlines(xaxis_idxes[obs_idx], -1, 1, lw=.5, color='w')
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    align_yaxis(ax, ax2, 0)
    
#     subplot_idx += 1
#     ax = plt.subplot(subplot_num,1,subplot_idx)
#     PlotTool.custom_ax(ax)
#     ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
#     for target_idx in target_idxes:
#         ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.2, color='w')
#     custom_axticks(ax, xaxis_idxes, x_axis_slice)
#     ax2 = ax.twinx()
#     PlotTool.custom_ax(ax2)
#     ax2.plot(xaxis_idxes, df.rsi_1h, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_1t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_2t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_3t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_4t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_5t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_6t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_7t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.rsi_8t, lw=.1, color='cyan')
#     ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
#     ax2.hlines(50, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
#     ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
#     min_y = min(min(df.rsi_8t), min(df.rsi_7t), min(df.rsi_6t), min(df.rsi_5t), min(df.rsi_4t), min(df.rsi_3t), min(df.rsi_2t), min(df.rsi_1t))
#     max_y = max(min(df.rsi_8t), min(df.rsi_7t), min(df.rsi_6t), max(df.rsi_5t), max(df.rsi_4t), max(df.rsi_3t), max(df.rsi_2t), max(df.rsi_1t))
#     custom_axticks(ax2, xaxis_idxes, x_axis_slice)
    
    
    
#     subplot_idx += 1
#     ax = plt.subplot(subplot_num,1,subplot_idx)
#     PlotTool.custom_ax(ax)
#     ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
#     for target_idx in target_idxes:
#         ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.2, color='w')
#     custom_axticks(ax, xaxis_idxes, x_axis_slice)
#     ax2 = ax.twinx()
#     PlotTool.custom_ax(ax2)
#     ax2.plot(xaxis_idxes, df.mfi_1t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_2t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_3t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_4t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_5t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_6t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_7t, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.mfi_8t, lw=.1, color='cyan')
#     ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
#     ax2.hlines(50, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
#     ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
#     custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ####################################
    ### MACD
    ####################################
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
    
    ### macd 1t cross 1h, all up
#     cond_macd_1t_corss_1h = (df.macdhist_1t > df.macdhist_1h) & (df.macdhist_1t.shift(1) < df.macdhist_1h.shift(1)) & ((df.macdhist_1t) > df.macdhist_1t.shift(1)) & ((df.macdhist_1h) > df.macdhist_1h.shift(1))
#     for cond_idx in df[cond_macd_1t_corss_1h].index:
#         ax.vlines(xaxis_idxes[cond_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
#     ax2.plot(xaxis_idxes, df.macdhist_4w, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_3w, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_2w, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_1w, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_4d, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_3d, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_2d, lw=.1, color='cyan')
#     ax2.plot(xaxis_idxes, df.macdhist_1d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.macdhist_1h, lw=.3, color='cyan')
    ax2.plot(xaxis_idxes, df.macdhist_1t, lw=.3, color='cyan')
    ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
#     min_y = min(min(df.macdhist_1t), min(df.macdhist_1h), min(df.macdhist_1d), min(df.macdhist_2d), min(df.macdhist_3d), min(df.macdhist_4d), min(df.macdhist_1w), min(df.macdhist_2w), min(df.macdhist_3w), min(df.macdhist_4w))
#     max_y = max(max(df.macdhist_1t), max(df.macdhist_1h), max(df.macdhist_1d), min(df.macdhist_2d), max(df.macdhist_3d), max(df.macdhist_4d), max(df.macdhist_1w), max(df.macdhist_2w), max(df.macdhist_3w), max(df.macdhist_4w))
    min_y = min(min(df.macdhist_1t), min(df.macdhist_1h))
    max_y = max(max(df.macdhist_1t), max(df.macdhist_1h))
    cond_macd_dlong = (df.macdhist_1d > df.macdhist_1d.shift(1)) & (df.macdhist_2d > df.macdhist_2d.shift(1)) & (df.macdhist_3d > df.macdhist_3d.shift(1)) & (df.macdhist_4d > df.macdhist_4d.shift(1))
    cond_macd_wlong = (df.macdhist_1w > df.macdhist_1w.shift(1)) & (df.macdhist_2w > df.macdhist_2w.shift(1)) & (df.macdhist_3w > df.macdhist_3w.shift(1)) & (df.macdhist_4w > df.macdhist_4w.shift(1))
    ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_dlong, facecolor='red',  alpha=0.3)
    ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_wlong, facecolor='lime',  alpha=0.3)
#     ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_long,   facecolor='lime', alpha=0.1)
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    ####################################
    ### K指标
    ####################################
    subplot_idx += 1
    ax = plt.subplot(subplot_num,1,subplot_idx)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
    cond_sp = (df.k_1t < 35) & (df.k_1h > 70) & (df.k_1d > 35) & (df.k_1d < 50)
    cond_sp = df.k_1d > 85
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_sp, facecolor='red',  alpha=0.2)
#     for target_idx in target_idxes:
    ax.vlines(xaxis_idxes[obs_idx], np.min(df.close), np.max(df.close), lw=.5, color='w')
    custom_axticks(ax, xaxis_idxes, x_axis_slice)
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.plot(xaxis_idxes, df.k_4w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_3w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_2w, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_5d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_4d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_3d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_2d, lw=.1, color='cyan')
    ax2.plot(xaxis_idxes, df.k_1d, lw=.5, color='cyan')
#     ax2.plot(xaxis_idxes, df.k_1h, lw=.5, color='cyan')
    ax2.plot(xaxis_idxes, df.k_1t, lw=.5, color='cyan')

#     minum_idxes = signal.argrelmin(df.k_1t.values, order=4)[0]
#     minimum_df = df.iloc[minum_idxes]
#     minimum_sub_df = minimum_df[(minimum_df.k_1t > minimum_df.k_1t.shift(1)) & (minimum_df.k_1t.shift(1) > minimum_df.k_1t.shift(2)) & (minimum_df.k_1t < 50) & (minimum_df.k_1d > 50)]
#     ax2.plot(minimum_sub_df.index, minimum_sub_df.k_1t, marker='.', lw=.2)
    
    ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
    ax2.hlines(50, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
    ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.1)
    ax2.fill_between(xaxis_idxes, df.k_1d, 50, where=df.k_1d < 50, facecolor='cyan', alpha=0.3)
    ax2.fill_between(xaxis_idxes, df.k_1d, 20, where=df.k_1d < 20, facecolor='cyan', alpha=0.3)
    ax2.fill_between(xaxis_idxes, 50, df.k_1d, where=df.k_1d > 50, facecolor='orange', alpha=0.3)
    ax2.fill_between(xaxis_idxes, 80, df.k_1d, where=df.k_1d > 80, facecolor='orange', alpha=0.3)
    custom_axticks(ax2, xaxis_idxes, x_axis_slice)

    plt.show()

## Save Match Target Figure

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()

delta = 15
daily_count = int(240/delta)

target_path = './target/'
target_fns = os.listdir(target_path)
target_fns.sort()
for target_fn in target_fns:
    if not target_fn.startswith('matches'):
        continue
    if os.path.isdir(target_path + target_fn):
        continue
    
    pic_path = target_path + target_fn[:-4] + '/'
    ts_jq = datetime.strptime(target_fn[8:-4], '%Y-%m-%d-%H%M')
    ts_jq = datetime.strftime(ts_jq, '%Y-%m-%d %H:%M:%S')
    if not os.path.exists(pic_path):
        os.mkdir(pic_path)
        target_df = pd.read_csv(target_path + target_fn, dtype={'sto_id':str, 'lastest_match_ts':str})
        for idx in target_df.index:
            try:
                record = target_df.iloc[idx]
                sto_id = record.sto_id
                target_ts = record.lastest_match_ts
                df = BaosAgent.read_freq_df(delta, sto_id)
                print('get realtime data from JQData ...')
                df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
                print('get realtime data from JQData accomplished.')
                
                factor_1t = daily_count * 1
                df['vol_rsi_1t'] = talib.RSI(df.volume, timeperiod=factor_1t)
                df['mfi_1t']     = talib.MFI(df.high, df.low, df.close, df.volume, timeperiod=factor_1t)
                df = add_macd_indicators(df)
                df['volrsi_mfi_prod'] = df.vol_rsi_1t * (df.mfi_1t - 50)
                df['volrsi_mfi_prod_roll1msum'] = df.volrsi_mfi_prod.rolling(daily_count*5*4).sum()
                df = df.dropna()
                df = df.iloc[-daily_count*5*8:]
                df = df.reset_index(drop=True)

                target_idx = df[df.time==target_ts].index.values[0]

                fig = plt.figure(figsize=(.5 * len(df)/daily_count, 10))
                fig.patch.set_facecolor('#2A2A2A')
                xaxis_idxes = df.index.values
                x_axis_slice = slice(0, -1, daily_count * 5)

                ax = plt.subplot(4,1,1)
                PlotTool.custom_ax(ax)
                ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
                ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.1, color='w')
                plt.sca(ax)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
                plt.yticks(fontsize=8, color='w')
                ax2 = ax.twinx()
                PlotTool.custom_ax(ax2)
                ax2.plot(xaxis_idxes, df.vol_rsi_1t, lw=.2, color='gold')
                ax2.plot(xaxis_idxes, df.mfi_1t,     lw=.2, color='lime')
                #     ax2.vlines(xaxis_idxes[target_idx], 0, 100, lw=.01, color='w')
                ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
                ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
                plt.sca(ax2)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
                plt.yticks(fontsize=8, color='w')



                ax = plt.subplot(4,1,2)
                PlotTool.custom_ax(ax)
                ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
                ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.1, color='w')
                plt.sca(ax)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
                plt.yticks(fontsize=8, color='w')
                ax2 = ax.twinx()
                PlotTool.custom_ax(ax2)
                ax2.plot(xaxis_idxes, df.volrsi_mfi_prod, lw=.2, color='gold')
                ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
                plt.sca(ax2)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
                plt.yticks(fontsize=8, color='w')



                ax = plt.subplot(4,1,3)
                PlotTool.custom_ax(ax)
                ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
                ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.1, color='w')
                plt.sca(ax)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
                plt.yticks(fontsize=8, color='w')
                ax2 = ax.twinx()
                PlotTool.custom_ax(ax2)
                ax2.plot(xaxis_idxes, df.volrsi_mfi_prod_roll1msum, lw=.2, color='gold')
                ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
                plt.sca(ax2)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
                plt.yticks(fontsize=8, color='w')



                ax = plt.subplot(4,1,4)
                PlotTool.custom_ax(ax)
                ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
                ax.vlines(xaxis_idxes[target_idx], np.min(df.close), np.max(df.close), lw=.1, color='w')
                plt.sca(ax)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
                plt.yticks(fontsize=8, color='w')
                ax2 = ax.twinx()
                PlotTool.custom_ax(ax2)
                ax2.plot(xaxis_idxes, df.macd_8t, lw=1, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_7t, lw=1, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_6t, lw=1, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_5t, lw=.2, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_4t, lw=.2, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_3t, lw=.2, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_2t, lw=.2, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_1t, lw=.2, color='cyan')
                ax2.plot(xaxis_idxes, df.macd_1h, lw=.2, color='cyan')
                ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.2)
                min_y = min(min(df.macd_5t), min(df.macd_4t), min(df.macd_3t), min(df.macd_2t), min(df.macd_1t), min(df.macd_1h))
                max_y = max(max(df.macd_5t), max(df.macd_4t), max(df.macd_3t), max(df.macd_2t), max(df.macd_1t), max(df.macd_1h))
                cond_macd_long = (df.macd_1t > df.macd_1t.shift(1)) & (df.macd_2t > df.macd_2t.shift(1)) & (df.macd_3t > df.macd_3t.shift(1)) & (df.macd_4t > df.macd_4t.shift(1)) & (df.macd_5t > df.macd_5t.shift(1)) & (df.macd_6t > df.macd_6t.shift(1)) & (df.macd_7t > df.macd_7t.shift(1)) & (df.macd_8t > df.macd_8t.shift(1)) & (df.macd_1h > df.macd_1h.shift(1))
                cond_macd_above0 = (df.macd_1h > 0) & (df.macd_1t > 0) & (df.macd_2t > 0) & (df.macd_3t > 0) & (df.macd_4t > 0) & (df.macd_5t > 0)
                ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_above0, facecolor='red', alpha=0.1)
                ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_long, facecolor='lime', alpha=0.1)
                plt.sca(ax2)
                plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
                plt.yticks(fontsize=8, color='w')

                curp = df.iloc[-1].close
                last_week_minp = min(df.close.iloc[-daily_count * 5:])
                last_week_cr = (curp - last_week_minp) / last_week_minp
                last_month_minp = min(df.close.iloc[-daily_count * 20:])
                last_month_cr = (curp - last_month_minp) / last_month_minp
                last_quart_minp = min(df.close.iloc[-daily_count * 60:])
                last_quart_cr = (curp - last_quart_minp) / last_quart_minp
                last_hyear_minp = min(df.close.iloc[-daily_count * 120:])
                last_hyear_cr = (curp - last_hyear_minp) / last_hyear_minp
                fn_fig = '{}_{:>5.2%}_{:>5.2%}_{:>5.2%}_{:>5.2%}.png'.format(sto_id, record.last_hyear_cr, record.last_quart_cr, record.last_month_cr, record.last_week_cr)
                print(fn_fig)
                plt.savefig(pic_path+fn_fig, dpi=512, bbox_inches='tight', facecolor='#2A2A2A', edgecolor='none')
                plt.close('all')
#                 clear_output()
            except Exception as e:
                print(sto_id, e)

jq.logout()

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()

jq.get_query_count()

## Observe data in REALTIME

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()

delta = 15
daily_count = int(240/delta)

target_path = './target/'
target_fns = os.listdir(target_path)
target_fns.sort()
for target_fn in target_fns:
    if not target_fn.startswith('matches'):
        continue
    if os.path.isdir(target_path + target_fn):
        continue
    
    print(target_fn[8:-4])
    target_df = pd.read_csv(target_path + target_fn, dtype={'sto_id':str, 'lastest_match_ts':str})
    for idx in target_df.index:
        record = target_df.iloc[idx]
        sto_id = record.sto_id
        close  = record.close
        ts     = record.lastest_match_ts
        
        jq_sto_id = JQDataUtil.get_jq_stoid(sto_id)
        df_rt = jq.get_bars(jq_sto_id, count=daily_count, unit=str(delta) + 'm',fields=['date', 'open','high','low','close','volume'],include_now=True,end_dt=JQDataUtil.get_end_ts(delta))
        df_rt.rename(columns={'date':'ts'}, inplace=True)
        df_rt.insert(0, 'code', sto_id)
        df_rt.insert(0, 'time', df_rt.ts.dt.strftime('%Y%m%d%H%M').values)
        df_rt.insert(0, 'date', df_rt.ts.dt.strftime('%Y-%m-%d').values)
        df_rt = df_rt[df_rt.time >= ts]
        df_rt = df_rt.reset_index(drop=True)
        cr = (df_rt.close.iloc[-1] - df_rt.close.iloc[0])/df_rt.close.iloc[0]
        print('{} {:>7.2%}'.format(sto_id, cr))
        print('{}\n'.format(df_rt.close.values))
    print('\n\n')

jq.logout()

In [ ]:
sto_id = random.sample(sto_ids, 1)[0]
df = BaosAgent.read_freq_df(15, sto_id)
l1 = len(df)
df = add_macd_indicators(df)
df = df.dropna()
l2 = len(df)
print(l1, l2, l1 - l2)

In [ ]:
ts

In [ ]:
jq_sto_id = JQDataUtil.get_jq_stoid(sto_id)
df_rt = jq.get_price(jq_sto_id, count = daily_count * 1, end_date=ts_jq, frequency=str(delta) + 'm', fields=['open','high','low','close','volume'])

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()

In [ ]:


jq_sto_id = JQDataUtil.get_jq_stoid(sto_id)
df_rt = jq.get_bars(jq_sto_id, count=daily_count, unit=str(delta) + 'm',fields=['date', 'open','high','low','close','volume'],include_now=True,end_dt=JQDataUtil.get_end_ts(delta))
df_rt



In [ ]:
df_rt.date.dt.strftime('%Y%m%d%H%M').values

In [ ]:

pd.Series(df_rt.date).dt.strftime('%Y%m%d%H%M').values

In [ ]:
df_rt.date.dt.strftime('%Y%m%d%H%M')

### observe crabove10in1week 

In [ ]:
all_crabove10in1week_df = pd.read_csv('all_crabove10in1week_df.csv', dtype={'sto_id':str, 'ts':str})

In [ ]:
plt.rcParams['figure.dpi'] = 300

In [ ]:
len(df[df.time==target_ts])

In [ ]:
df = BaosAgent.read_freq_df(delta, target_sto_id)
factor_1t = daily_count * 1
df['vol_rsi_1t'] = talib.RSI(df.volume, timeperiod=factor_1t)
df['mfi_1t']     = talib.MFI(df.high, df.low, df.close, df.volume, timeperiod=factor_1t)
df = add_macd_indicators(df)

df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
print(target_sto_id, target_ts)

In [ ]:
df.time.iloc[0], df.time.iloc[-1]

In [ ]:


# df['close_max_in1week'] = df.close.rolling(daily_count * 5).max().shift(-daily_count * 5)
# df['close_min_in1week'] = df.close.rolling(daily_count * 5).min().shift(-daily_count * 5)
# df['cr_max_in1week']    = (df.close_max_in1week - df.close) / df.close
# df['cr_min_in1week']    = (df.close_min_in1week - df.close) / df.close

df = add_kds_indicators(df)
df = add_volrsi_indicators(df)
df = add_mfi_indicators(df)
df = add_macd_indicators(df)
df['close_1week_mean']   = df.close.rolling(daily_count * 5).mean()
df['close_1week_median'] = df.close.rolling(daily_count * 5).median()
df['close_1week_std']    = df.close.rolling(daily_count * 5).std()
df['CLOSE_MA_18d'] = talib.MA(df.close,  timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close,  timeperiod=daily_count * 50)
df['VOL_MA_18d']   = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d']   = talib.MA(df.volume, timeperiod=daily_count * 50)
df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)
df['vol_rolling1d_times'] = df.volume / df.volume.rolling(daily_count).mean()
df['vol_rolling1w_times'] = df.volume / df.volume.rolling(daily_count*5).mean()
df['vol_rolling1m_times'] = df.volume / df.volume.rolling(daily_count*20).mean()


df = df.dropna()
df = df.iloc[-daily_count * 5 * 8:].copy()
df = df.reset_index(drop=True)

# cond = (df.cr_max_in1week > 0.1) & (df.cr_min_in1week > 0)


fig = plt.figure(figsize=(.5 * len(df)/daily_count, 6))
fig.patch.set_facecolor('#2A2A2A')

xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)


ax = plt.subplot(2,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')

ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
# ax2.plot(xaxis_idxes, df.k_9t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_8t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_7t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_6t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_5t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_4t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_3t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_2t, lw=.1, color='cyan')
# ax.plot(xaxis_idxes, df.k_1t, lw=.1, color='cyan')


# ax2.plot(xaxis_idxes, df.vol_rsi_9t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_8t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_7t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_6t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_5t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_4t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_3t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_2t, lw=.2, color='gold')
ax2.plot(xaxis_idxes, df.vol_rsi_1t, lw=.2, color='gold')

# ax2.plot(xaxis_idxes, df.mfi_9t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_8t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_7t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_6t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_5t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_4t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_3t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_2t, lw=.2, color='lime')
ax2.plot(xaxis_idxes, df.mfi_1t, lw=.2, color='lime')

# cond = (df.cr_max_in1week > 0.1) & (df.cr_min_in1week > 0)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], 0, 100, lw=.2, color='w')

ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)
ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)

plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

ax = plt.subplot(2,1,2)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_5t, lw=1 , color='cyan')
ax2.plot(xaxis_idxes, df.macd_4t, lw=.8, color='cyan')
ax2.plot(xaxis_idxes, df.macd_3t, lw=.6, color='cyan')
ax2.plot(xaxis_idxes, df.macd_2t, lw=.4, color='cyan')
ax2.plot(xaxis_idxes, df.macd_1t, lw=.2, color='cyan')
ax2.plot(xaxis_idxes, df.macd_1h, lw=.1, color='cyan')

# ax2.plot(xaxis_idxes, df.macdsignal_5t, lw=1 , color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_4t, lw=.8, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_3t, lw=.6, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_2t, lw=.4, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_1t, lw=.2, color='gold')


ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
min_y = min(min(df.macd_5t), min(df.macd_4t), min(df.macd_3t), min(df.macd_2t), min(df.macd_1t))
max_y = max(max(df.macd_5t), max(df.macd_4t), max(df.macd_3t), max(df.macd_2t), max(df.macd_1t))
cond_macd_long = (df.macd_1t > df.macd_1t.shift(1)) & (df.macd_2t > df.macd_2t.shift(1)) & (df.macd_3t > df.macd_3t.shift(1)) & (df.macd_4t > df.macd_4t.shift(1)) & (df.macd_5t > df.macd_5t.shift(1)) & (df.macd_1h > df.macd_1h.shift(1))
ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_long, facecolor='red', alpha=0.1)

# for tgt_idx in df[cond].index:
#     ax2.vlines(xaxis_idxes[tgt_idx], min_y, max_y, lw=.01, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

plt.show()

### Observe Long Periods

In [ ]:
def add_ma_subplot(num_subplots, idx_subplot, df):
    xaxis_idxes = df.index.values
    x_axis_slice = slice(0, -1, daily_count * 5)
    
    cond_ma_long  = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
    cond_ma_short = (df.CLOSE_MA_18d < df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d < df.CLOSE_MA_50d.shift(1))
    cond_ma_consolidation = ((df.CLOSE_MA_18d <= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d >= df.CLOSE_MA_50d.shift(1))) | \
                            ((df.CLOSE_MA_18d >= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d <= df.CLOSE_MA_50d.shift(1)))

    ax = plt.subplot(num_subplots,1,idx_subplot)
    PlotTool.custom_ax(ax)
    ax.plot(xaxis_idxes, df.CLOSE_MA_18d, lw=.5, color='cyan')
    ax.plot(xaxis_idxes, df.CLOSE_MA_50d, lw=.5, color='lime')
    ax.plot(xaxis_idxes, df.close,        lw=.5, color='orange')
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_long,          facecolor='red',    alpha=0.1)
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_short,         facecolor='lime',   alpha=0.1)
    ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_ma_consolidation, facecolor='orange', alpha=0.1)
    
    df['vol_rolling_4week_mean'] = df.volume.rolling(daily_count*20).mean()
    cond_bband_long_cross = (df.close > df.BBAND_upper) & (df.close.shift(1) < df.BBAND_upper.shift(1))
    cond_vol_rise         = (df.volume > df.vol_rolling_4week_mean * 2)
    cond = cond_bband_long_cross & cond_vol_rise & cond_ma_long
    for tgt_idx in df[cond].index:
        cur_p = df.iloc[tgt_idx].close
        max_p = np.max(df.close.iloc[tgt_idx + 1:tgt_idx+daily_count*5])
        min_p = np.min(df.close.iloc[tgt_idx + 1:tgt_idx+daily_count*5])
        max_cr = (max_p - cur_p) / cur_p
        min_cr = (min_p - cur_p) / cur_p
        print('{:>6} {:>7.2%} {:>7.2%} {:>5.2f}'.format(tgt_idx, max_cr, min_cr, df.volume.iloc[tgt_idx] / df.vol_rolling_4week_mean.iloc[tgt_idx]))
#     for tgt_idx in df[cond].index:
        ax.vlines(xaxis_idxes[tgt_idx], np.min(df.close), np.max(df.close), lw=1, color='w')
    
    plt.sca(ax)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
    plt.yticks(fontsize=8, color='w')
    
    ax2 = ax.twinx()
    PlotTool.custom_ax(ax2)
    ax2.fill_between(xaxis_idxes, 0, df.volume/100, facecolor='cyan')
    plt.sca(ax2)
    plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
    plt.yticks(fontsize=8, color='w')

In [ ]:
delta = 15
daily_count = int(240/delta)

sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]
print(sto_id)
df = BaosAgent.read_freq_df(delta, sto_id)
# print('get realtime data from JQData ...')
# df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
# print('get realtime data from JQData accomplished.')
df = add_kds_indicators(df)
df['CLOSE_MA_18d'] = talib.MA(df.close,  timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close,  timeperiod=daily_count * 50)
df['VOL_MA_18d']   = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d']   = talib.MA(df.volume, timeperiod=daily_count * 50)
df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

df = df.dropna()
df = df.iloc[-daily_count * 5 * 32:]
df = df.reset_index(drop=True)

cond_long_9t = compute_cond_long(df.k_9t, df.d_9t)
cond_long_8t = compute_cond_long(df.k_8t, df.d_8t)
cond_long_7t = compute_cond_long(df.k_7t, df.d_7t)
cond_long_6t = compute_cond_long(df.k_6t, df.d_6t)
cond_long_5t = compute_cond_long(df.k_5t, df.d_5t)
cond_long_4t = compute_cond_long(df.k_4t, df.d_4t)
cond_long_3t = compute_cond_long(df.k_3t, df.d_3t)
cond_long_2t = compute_cond_long(df.k_2t, df.d_2t)
cond_long_1t = compute_cond_long(df.k_1t, df.d_1t)
cond_all_long = cond_long_1t & cond_long_2t & cond_long_3t & cond_long_4t & cond_long_5t & cond_long_6t & cond_long_7t# & cond_long_8t & cond_long_9t

cond_short_9t = compute_cond_short(df.k_9t, df.d_9t)
cond_short_8t = compute_cond_short(df.k_8t, df.d_8t)
cond_short_7t = compute_cond_short(df.k_7t, df.d_7t)
cond_short_6t = compute_cond_short(df.k_6t, df.d_6t)
cond_short_5t = compute_cond_short(df.k_5t, df.d_5t)
cond_short_4t = compute_cond_short(df.k_4t, df.d_4t)
cond_short_3t = compute_cond_short(df.k_3t, df.d_3t)
cond_short_2t = compute_cond_short(df.k_2t, df.d_2t)
cond_short_1t = compute_cond_short(df.k_1t, df.d_1t)

cond_high_saturation_9t = compute_high_saturation(df.k_9t)
cond_high_saturation_8t = compute_high_saturation(df.k_8t)
cond_high_saturation_7t = compute_high_saturation(df.k_7t)
cond_high_saturation_6t = compute_high_saturation(df.k_6t)
cond_high_saturation_5t = compute_high_saturation(df.k_5t)
cond_high_saturation_4t = compute_high_saturation(df.k_4t)
cond_high_saturation_3t = compute_high_saturation(df.k_3t)
cond_high_saturation_2t = compute_high_saturation(df.k_2t)
cond_high_saturation_1t = compute_high_saturation(df.k_1t)

cond_low_saturation_9t = compute_low_saturation(df.k_9t)
cond_low_saturation_8t = compute_low_saturation(df.k_8t)
cond_low_saturation_7t = compute_low_saturation(df.k_7t)
cond_low_saturation_6t = compute_low_saturation(df.k_6t)
cond_low_saturation_5t = compute_low_saturation(df.k_5t)
cond_low_saturation_4t = compute_low_saturation(df.k_4t)
cond_low_saturation_3t = compute_low_saturation(df.k_3t)
cond_low_saturation_2t = compute_low_saturation(df.k_2t)
cond_low_saturation_1t = compute_low_saturation(df.k_1t)

num_subplots, idx_subplot = 2, 0
fig = plt.figure(figsize=(.5 * len(df)/daily_count, num_subplots * 1.75))
fig.patch.set_facecolor('#2A2A2A')

idx_subplot += 1
add_ma_subplot(num_subplots, idx_subplot, df)
idx_subplot += 1
add_bband_subplot(num_subplots, idx_subplot, df)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_1t, df.d_1t, cond_high_saturation_1t, cond_low_saturation_1t, cond_long_1t, cond_short_1t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_2t, df.d_2t, cond_high_saturation_2t, cond_low_saturation_2t, cond_long_2t, cond_short_2t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_3t, df.d_3t, cond_high_saturation_3t, cond_low_saturation_3t, cond_long_3t, cond_short_3t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_4t, df.d_4t, cond_high_saturation_4t, cond_low_saturation_4t, cond_long_4t, cond_short_4t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_5t, df.d_5t, cond_high_saturation_5t, cond_low_saturation_5t, cond_long_5t, cond_short_5t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_6t, df.d_6t, cond_high_saturation_6t, cond_low_saturation_6t, cond_long_6t, cond_short_6t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_7t, df.d_7t, cond_high_saturation_7t, cond_low_saturation_7t, cond_long_7t, cond_short_7t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_8t, df.d_8t, cond_high_saturation_8t, cond_low_saturation_8t, cond_long_8t, cond_short_8t, df.close, cond_all_long)
# idx_subplot += 1
# add_kd_subplot(num_subplots, idx_subplot, df.k_9t, df.d_9t, cond_high_saturation_9t, cond_low_saturation_9t, cond_long_9t, cond_short_9t, df.close, cond_all_long)

curp = df.iloc[-1].close
last_week_minp = min(df.close.iloc[-daily_count * 5:])
last_week_cr = (curp - last_week_minp) / last_week_minp
last_month_minp = min(df.close.iloc[-daily_count * 20:])
last_month_cr = (curp - last_month_minp) / last_month_minp
last_quart_minp = min(df.close.iloc[-daily_count * 60:])
last_quart_cr = (curp - last_quart_minp) / last_quart_minp
last_hyear_minp = min(df.close.iloc[-daily_count * 120:])
last_hyear_cr = (curp - last_hyear_minp) / last_hyear_minp

plt.show()

In [ ]:
delta = 15
daily_count = int(240/delta)

sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(match_sto_infos, 1)[0]
print(sto_id)
df = BaosAgent.read_freq_df(delta, sto_id)

# df['close_max_in1week'] = df.close.rolling(daily_count * 5).max().shift(-daily_count * 5)
# df['close_min_in1week'] = df.close.rolling(daily_count * 5).min().shift(-daily_count * 5)
# df['cr_max_in1week']    = (df.close_max_in1week - df.close) / df.close
# df['cr_min_in1week']    = (df.close_min_in1week - df.close) / df.close

df = add_kds_indicators(df)
df = add_volrsi_indicators(df)
df = add_mfi_indicators(df)
df = add_macd_indicators(df)
df['close_1week_mean']   = df.close.rolling(daily_count * 5).mean()
df['close_1week_median'] = df.close.rolling(daily_count * 5).median()
df['close_1week_std']    = df.close.rolling(daily_count * 5).std()
df['CLOSE_MA_18d'] = talib.MA(df.close,  timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close,  timeperiod=daily_count * 50)
df['VOL_MA_18d']   = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d']   = talib.MA(df.volume, timeperiod=daily_count * 50)
df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)
df['vol_rolling1d_times'] = df.volume / df.volume.rolling(daily_count).mean()
df['vol_rolling1w_times'] = df.volume / df.volume.rolling(daily_count*5).mean()
df['vol_rolling1m_times'] = df.volume / df.volume.rolling(daily_count*20).mean()


df = df.dropna()
df = df.iloc[-daily_count * 5 * 8:].copy()
df = df.reset_index(drop=True)

# cond = (df.cr_max_in1week > 0.1) & (df.cr_min_in1week > 0)


fig = plt.figure(figsize=(.5 * len(df)/daily_count, 6))
fig.patch.set_facecolor('#2A2A2A')

xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)


ax = plt.subplot(2,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')

ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
# ax2.plot(xaxis_idxes, df.k_9t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_8t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_7t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_6t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_5t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_4t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_3t, lw=.1, color='cyan')
# ax2.plot(xaxis_idxes, df.k_2t, lw=.1, color='cyan')
# ax.plot(xaxis_idxes, df.k_1t, lw=.1, color='cyan')


# ax2.plot(xaxis_idxes, df.vol_rsi_9t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_8t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_7t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_6t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_5t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_4t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_3t, lw=.2, color='gold')
# ax2.plot(xaxis_idxes, df.vol_rsi_2t, lw=.2, color='gold')
ax2.plot(xaxis_idxes, df.vol_rsi_1t, lw=.2, color='gold')

# ax2.plot(xaxis_idxes, df.mfi_9t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_8t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_7t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_6t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_5t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_4t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_3t, lw=.2, color='lime')
# ax2.plot(xaxis_idxes, df.mfi_2t, lw=.2, color='lime')
ax2.plot(xaxis_idxes, df.mfi_1t, lw=.2, color='lime')

# cond = (df.cr_max_in1week > 0.1) & (df.cr_min_in1week > 0)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], 0, 100, lw=.2, color='w')

ax2.hlines(20, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)
ax2.hlines(80, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=.5)

plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

ax = plt.subplot(2,1,2)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_5t, lw=1 , color='cyan')
ax2.plot(xaxis_idxes, df.macd_4t, lw=.8, color='cyan')
ax2.plot(xaxis_idxes, df.macd_3t, lw=.6, color='cyan')
ax2.plot(xaxis_idxes, df.macd_2t, lw=.4, color='cyan')
ax2.plot(xaxis_idxes, df.macd_1t, lw=.2, color='cyan')
ax2.plot(xaxis_idxes, df.macd_1h, lw=.1, color='cyan')

# ax2.plot(xaxis_idxes, df.macdsignal_5t, lw=1 , color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_4t, lw=.8, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_3t, lw=.6, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_2t, lw=.4, color='gold')
# ax2.plot(xaxis_idxes, df.macdsignal_1t, lw=.2, color='gold')


ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
min_y = min(min(df.macd_5t), min(df.macd_4t), min(df.macd_3t), min(df.macd_2t), min(df.macd_1t))
max_y = max(max(df.macd_5t), max(df.macd_4t), max(df.macd_3t), max(df.macd_2t), max(df.macd_1t))
cond_macd_long = (df.macd_1t > df.macd_1t.shift(1)) & (df.macd_2t > df.macd_2t.shift(1)) & (df.macd_3t > df.macd_3t.shift(1)) & (df.macd_4t > df.macd_4t.shift(1)) & (df.macd_5t > df.macd_5t.shift(1)) & (df.macd_1h > df.macd_1h.shift(1))
ax2.fill_between(xaxis_idxes, min_y, max_y, where=cond_macd_long, facecolor='red', alpha=0.1)

# for tgt_idx in df[cond].index:
#     ax2.vlines(xaxis_idxes[tgt_idx], min_y, max_y, lw=.01, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

plt.show()

In [ ]:
df.iloc[-2:][['macd_1h', 'macd_1t', 'macd_2t', 'macd_3t', 'macd_4t', 'macd_5t']]

In [ ]:
cond_macd_long.iloc[-1]

In [ ]:

cond = (df.mfi_1t > 50).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
df[cond].index

In [ ]:
fig = plt.figure(figsize=(.3 * len(df)/daily_count, 12))
fig.patch.set_facecolor('#2A2A2A')

xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)

ax = plt.subplot(5,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_1t,       lw=.5, color='cyan')
ax2.plot(xaxis_idxes, df.macdsignal_1t, lw=.5, color='gold')

ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], min(df.macd_5t), max(df.macd_1t), lw=.2, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

ax = plt.subplot(5,1,2)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_2t,       lw=.5, color='cyan')
ax2.plot(xaxis_idxes, df.macdsignal_2t, lw=.5, color='gold')
ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], min(df.macd_5t), max(df.macd_1t), lw=.2, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')


ax = plt.subplot(5,1,3)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_3t,       lw=.5, color='cyan')
ax2.plot(xaxis_idxes, df.macdsignal_3t, lw=.5, color='gold')
ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], min(df.macd_5t), max(df.macd_1t), lw=.2, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')


ax = plt.subplot(5,1,4)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_4t,       lw=.5, color='cyan')
ax2.plot(xaxis_idxes, df.macdsignal_4t, lw=.5, color='gold')
ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], min(df.macd_5t), max(df.macd_1t), lw=.2, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

ax = plt.subplot(5,1,5)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')
ax2 = ax.twinx()
PlotTool.custom_ax(ax2)
ax2.plot(xaxis_idxes, df.macd_5t,       lw=.5, color='cyan')
ax2.plot(xaxis_idxes, df.macdsignal_5t, lw=.5, color='gold')
ax2.hlines(0, xmin=min(xaxis_idxes), xmax=max(xaxis_idxes), color='w', lw=1)
for tgt_idx in df[cond].index:
    ax2.vlines(xaxis_idxes[tgt_idx], min(df.macd_5t), max(df.macd_1t), lw=.2, color='w')
plt.sca(ax2)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w', alpha=.3)
plt.yticks(fontsize=8, color='w')

plt.show()

In [ ]:
mfi_mean, mfi_median, mfi_std = df.mfi_1t.iloc[900:980].mean(), df.mfi_1t.iloc[900:980].median(), df.mfi_1t.iloc[900:980].std()
mfi_zscore = (df.iloc[980].mfi_1t - mfi_mean) / mfi_std
print('{:>6.2f} {:>6.2f} {:>6.2f} {:>6.2f}'.format(mfi_mean, mfi_median, mfi_std, mfi_zscore))

In [ ]:
close_mean, close_median, close_std = df.close.iloc[900:980].mean(), df.close.iloc[900:980].median(), df.close.iloc[900:980].std()
close_zscore = (df.iloc[980].close - close_mean) / close_std
print('{:>6.2f} {:>6.2f} {:>6.2f} {:>6.2f}'.format(close_mean, close_median, close_std, close_zscore))

In [ ]:
delta = 15
daily_count = int(240/delta)

sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]

df = BaosAgent.read_freq_df(delta, sto_id)
df = add_kds_indicators(df)
df['CLOSE_MA_18d'] = talib.MA(df.close,  timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close,  timeperiod=daily_count * 50)
df['VOL_MA_18d']   = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d']   = talib.MA(df.volume, timeperiod=daily_count * 50)
df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

cond_ma_long          = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
cond_bband_long_cross = (df.close > df.BBAND_upper) & (df.close.shift(1) <= df.BBAND_upper.shift(1))
cond_vol_rise         = (df.volume > df.volume.rolling(daily_count*5).mean() * 2)
cond = cond_ma_long & cond_bband_long & cond_vol_rise
for idx in df[cond].index:
    cur_p = df.iloc[idx].close
    max_p = np.max(df.close.iloc[idx + 1:idx+daily_count*5])
    min_p = np.min(df.close.iloc[idx + 1:idx+daily_count*5])
    max_cr = (max_p - cur_p) / cur_p
    min_cr = (min_p - cur_p) / cur_p
    print('{:>6} {:>6.2%} {:>6.2%}'.format(idx, max_cr, min_cr))

In [ ]:
datas = []
for idx in range(len(max_crs)):
    datas.append([max_crs[idx], min_crs[idx]])
crs_df = pd.DataFrame(columns=['max_cr', 'min_cr'], data=datas)

In [ ]:
crs_df

In [ ]:
crs_df.max_cr.hist(bins=1000)

In [ ]:
crs_df.min_cr.hist(bins=1000)

In [ ]:
print('{:>7.2%} {:>7.2%} {:>7.2%} {:>7.2%}'.format(np.nanmin(max_crs), np.nanmax(max_crs), np.nanmean(max_crs), np.nanstd(max_crs)))
print('{:>7.2%} {:>7.2%} {:>7.2%} {:>7.2%}'.format(np.nanmin(min_crs), np.nanmax(min_crs), np.nanmean(min_crs), np.nanstd(min_crs)))

In [ ]:
len(crs_df[crs_df.max_cr > .05]), len(crs_df)

In [ ]:
delta = 15
sto_id = random.sample(sto_ids, 1)[0]
all_datas = []
for sto_id in tqdm(sto_ids):
    df = BaosAgent.read_freq_df(delta, sto_id)
    df = add_kds_indicators(df)
    df = df.dropna()
    df = df.reset_index(drop=True)
    datas = loc_lower_deviate(df.k_3t, df.d_3t)
    for data in datas:
        all_datas.append(data)
print(len(all_datas))

In [ ]:
all_datas_df = pd.DataFrame(columns=['sto_id', 'last_dcross_k', 'gcross_k', 'last_dcross_close', 'gcross_close', 'max_cr', 'min_cr'], data=all_datas)

In [ ]:
all_datas_df.corrwith(all_datas_df.max_cr)

In [ ]:
def loc_lower_deviate(k, d):
    cond_gcross = (k > d) & (k.shift(1) < d.shift(1))
    cond_dcross = (k < d) & (k.shift(1) > d.shift(1))
    gcross_idxes = df[cond_gcross].index
    dcross_idxes = df[cond_dcross].index
    datas = []
    for gcross_idx in gcross_idxes:
        if len(dcross_idxes[dcross_idxes < gcross_idx]) == 0:
            continue
        last_dcross_idx = dcross_idxes[dcross_idxes < gcross_idx][-1]
        last_dcross_k, gcross_k = k.iloc[last_dcross_idx], k.iloc[gcross_idx]
        last_dcross_close, gcross_close = df.close.iloc[last_dcross_idx], df.close.iloc[gcross_idx]
        if last_dcross_k < gcross_k and last_dcross_close > gcross_close:
            max_p = np.max(df.high.iloc[gcross_idx + 1 : gcross_idx + daily_count * 5])
            min_p = np.min(df.low.iloc[gcross_idx + 1 : gcross_idx + daily_count * 5])
            max_cr = (max_p - gcross_close)/gcross_close
            min_cr = (min_p - gcross_close)/gcross_close
            print('{} {:>6.2%} {:>6.2%}'.format(sto_id, max_cr, min_cr))
            datas.append([sto_id, last_dcross_k, gcross_k, last_dcross_close, gcross_close, max_cr, min_cr])
    return datas

In [ ]:
def loc_lower_deviate(k, d):
    cond_gcross = (k > d) & (k.shift(1) < d.shift(1))
    cond_dcross = (k < d) & (k.shift(1) > d.shift(1))
    gcross_idxes = df[cond_gcross].index
    dcross_idxes = df[cond_dcross].index
    datas = []
    for gcross_idx in gcross_idxes:
        if len(dcross_idxes[dcross_idxes < gcross_idx]) == 0:
            continue
        last_dcross_idx = dcross_idxes[dcross_idxes < gcross_idx][-1]
        last_dcross_k, gcross_k = k.iloc[last_dcross_idx], k.iloc[gcross_idx]
        last_dcross_close, gcross_close = df.close.iloc[last_dcross_idx], df.close.iloc[gcross_idx]
        if last_dcross_k < gcross_k and last_dcross_close > gcross_close:
            max_p = np.max(df.high.iloc[gcross_idx + 1 : gcross_idx + daily_count * 5])
            min_p = np.min(df.low.iloc[gcross_idx + 1 : gcross_idx + daily_count * 5])
            max_cr = (max_p - gcross_close)/gcross_close
            min_cr = (min_p - gcross_close)/gcross_close
            print('{} {:>6.2%} {:>6.2%}'.format(sto_id, max_cr, min_cr))
            datas.append([sto_id, last_dcross_k, gcross_k, last_dcross_close, gcross_close, max_cr, min_cr])
    return datas

In [ ]:
# jq.auth('15808061188', 'allan2jq')
# clear_output()
# jq.get_query_count()

delta = 15
daily_count = int(240/delta)

sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]

# sto_id = '002557'

# sto_id = random.sample(sto_ids, 1)[0]

# matches_df = pd.read_csv('./target/matches_2020-02-08-1500.csv', dtype={'sto_id':str})
# sto_ids_backup = matches_df.sto_id.tolist()
# sto_ids_backup.sort()
# sto_id = random.sample(sto_ids_backup, 1)[0]



# for sto_id in sto_ids:
df = BaosAgent.read_freq_df(delta, sto_id)
# print('get data from baostock ...')
# df = BaosAgent.get_freq_data_from_bs(sto_id, '2014-01-01', delta)
# print('get data from baostock accomplished.')
# df.time = df.time.str[:-5]
# df.code = df.code.str[3:]
# df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
# df.open = np.round(df.open, 2)
# df.high = np.round(df.high, 2)
# df.low = np.round(df.low, 2)
# df.close = np.round(df.close, 2)

# print('get realtime data from JQData ...')
# df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
# print('get realtime data from JQData accomplished.')

org_size = len(df)
if org_size > 799:
    print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])

    daily_count = int(240/delta)
    df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
    df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)
    df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
    df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)
    df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

    df = add_kds_indicators(df)

    df = df.dropna()

    print(org_size - len(df))
    df = df.iloc[-daily_count * 5 * 32:]
    df = df.reset_index(drop=True)
    print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])

    cond_long_9t = compute_cond_long(df.k_9t, df.d_9t)
    cond_long_8t = compute_cond_long(df.k_8t, df.d_8t)
    cond_long_7t = compute_cond_long(df.k_7t, df.d_7t)
    cond_long_6t = compute_cond_long(df.k_6t, df.d_6t)
    cond_long_5t = compute_cond_long(df.k_5t, df.d_5t)
    cond_long_4t = compute_cond_long(df.k_4t, df.d_4t)
    cond_long_3t = compute_cond_long(df.k_3t, df.d_3t)
    cond_long_2t = compute_cond_long(df.k_2t, df.d_2t)
    cond_long_1t = compute_cond_long(df.k_1t, df.d_1t)
    cond_all_long = cond_long_1t & cond_long_2t & cond_long_3t & cond_long_4t & cond_long_5t & cond_long_6t & cond_long_7t & cond_long_8t & cond_long_9t
    
    cond_short_9t = compute_cond_short(df.k_9t, df.d_9t)
    cond_short_8t = compute_cond_short(df.k_8t, df.d_8t)
    cond_short_7t = compute_cond_short(df.k_7t, df.d_7t)
    cond_short_6t = compute_cond_short(df.k_6t, df.d_6t)
    cond_short_5t = compute_cond_short(df.k_5t, df.d_5t)
    cond_short_4t = compute_cond_short(df.k_4t, df.d_4t)
    cond_short_3t = compute_cond_short(df.k_3t, df.d_3t)
    cond_short_2t = compute_cond_short(df.k_2t, df.d_2t)
    cond_short_1t = compute_cond_short(df.k_1t, df.d_1t)
    
    cond_high_saturation_9t = compute_high_saturation(df.k_9t)
    cond_high_saturation_8t = compute_high_saturation(df.k_8t)
    cond_high_saturation_7t = compute_high_saturation(df.k_7t)
    cond_high_saturation_6t = compute_high_saturation(df.k_6t)
    cond_high_saturation_5t = compute_high_saturation(df.k_5t)
    cond_high_saturation_4t = compute_high_saturation(df.k_4t)
    cond_high_saturation_3t = compute_high_saturation(df.k_3t)
    cond_high_saturation_2t = compute_high_saturation(df.k_2t)
    cond_high_saturation_1t = compute_high_saturation(df.k_1t)
    
    cond_low_saturation_9t = compute_low_saturation(df.k_9t)
    cond_low_saturation_8t = compute_low_saturation(df.k_8t)
    cond_low_saturation_7t = compute_low_saturation(df.k_7t)
    cond_low_saturation_6t = compute_low_saturation(df.k_6t)
    cond_low_saturation_5t = compute_low_saturation(df.k_5t)
    cond_low_saturation_4t = compute_low_saturation(df.k_4t)
    cond_low_saturation_3t = compute_low_saturation(df.k_3t)
    cond_low_saturation_2t = compute_low_saturation(df.k_2t)
    cond_low_saturation_1t = compute_low_saturation(df.k_1t)

    num_subplots, idx_subplot = 11, 0
    fig = plt.figure(figsize=(.2 * len(df)/daily_count, num_subplots * 1.75))
    fig.patch.set_facecolor('#2A2A2A')

    idx_subplot += 1
    add_ma_subplot(num_subplots, idx_subplot, df)

    idx_subplot += 1
    add_bband_subplot(num_subplots, idx_subplot, df)

    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_1t, df.d_1t, cond_high_saturation_1t, cond_low_saturation_1t, cond_long_1t, cond_short_1t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_2t, df.d_2t, cond_high_saturation_2t, cond_low_saturation_2t, cond_long_2t, cond_short_2t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_3t, df.d_3t, cond_high_saturation_3t, cond_low_saturation_3t, cond_long_3t, cond_short_3t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_4t, df.d_4t, cond_high_saturation_4t, cond_low_saturation_4t, cond_long_4t, cond_short_4t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_5t, df.d_5t, cond_high_saturation_5t, cond_low_saturation_5t, cond_long_5t, cond_short_5t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_6t, df.d_6t, cond_high_saturation_6t, cond_low_saturation_6t, cond_long_6t, cond_short_6t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_7t, df.d_7t, cond_high_saturation_7t, cond_low_saturation_7t, cond_long_7t, cond_short_7t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_8t, df.d_8t, cond_high_saturation_8t, cond_low_saturation_8t, cond_long_8t, cond_short_8t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_9t, df.d_9t, cond_high_saturation_9t, cond_low_saturation_9t, cond_long_9t, cond_short_9t, df.close, cond_all_long)

    plt.show()
    plt.close('all')

    
# jq.logout()


In [ ]:
def compute_k_above60_sum(x):
    return np.sum(x[x > 60])

def compute_k_below40_sum(x):
    return np.sum(x[x < 40])

### Save Figure

In [ ]:
delta = 15
daily_count = int(240/delta)
factor_qut = daily_count * 64
factor_mon = daily_count * 16
factor_wek = daily_count * 4
factor_day = daily_count * 1
factor_60m = 4
factor_30m = 2
factor_15m = 1

sto_ids = get_freq_file_sto_ids(delta)

def find_targets(sub_sto_ids):
    match_sto_ids = []
    for sto_id in tqdm(sub_sto_ids):
        df = BaosAgent.read_freq_df(delta, sto_id)
        org_size = len(df)
        if org_size > daily_count * 120:
            curp = df.iloc[-1].close
            last_week_minp = min(df.close.iloc[-daily_count * 5:])
            last_week_cr = (curp - last_week_minp) / last_week_minp
            last_month_minp = min(df.close.iloc[-daily_count * 20:])
            last_month_cr = (curp - last_month_minp) / last_month_minp
            last_quart_minp = min(df.close.iloc[-daily_count * 60:])
            last_quart_cr = (curp - last_quart_minp) / last_quart_minp
            last_hyear_minp = min(df.close.iloc[-daily_count * 120:])
            last_hyear_cr = (curp - last_hyear_minp) / last_hyear_minp
            
            ### reduce compute data size ###
            df = df.iloc[-900:].copy()
            ################################
            df = add_kds_indicators(df)
            cond_long_9t = compute_cond_long(df.k_9t, df.d_9t)
            cond_long_8t = compute_cond_long(df.k_8t, df.d_8t)
            cond_long_7t = compute_cond_long(df.k_7t, df.d_7t)
            cond_long_6t = compute_cond_long(df.k_6t, df.d_6t)
            cond_long_5t = compute_cond_long(df.k_5t, df.d_5t)
            cond_long_4t = compute_cond_long(df.k_4t, df.d_4t)
            cond_long_3t = compute_cond_long(df.k_3t, df.d_3t)
            cond_long_2t = compute_cond_long(df.k_2t, df.d_2t)
            cond_long_1t = compute_cond_long(df.k_1t, df.d_1t)
            cond_all_long = cond_long_1t & cond_long_2t & cond_long_3t & cond_long_4t & cond_long_5t & cond_long_6t & cond_long_7t & cond_long_8t & cond_long_9t
            df = df.dropna()
            
            df = df.reset_index(drop=True)
            target_idx = -9
            if not (cond_all_long.iloc[target_idx - 1]) and cond_all_long.iloc[target_idx]:
                lastest_p = df.close.iloc[-1]
                target_p = df.close.iloc[target_idx]
                target_t = df.time.iloc[target_idx]
                print('{} {:>5} {:>5.2f} {:>5.2f} {:>5.2f} {:>7.2%} {:>7.2%} {:>7.2%} {:>7.2%}'.format(sto_id, len(df), target_p, lastest_p, lastest_p - target_p, last_hyear_cr, last_quart_cr, last_month_cr, last_week_cr))
                match_sto_ids.append([sto_id, target_t, target_p, lastest_p, last_hyear_cr, last_quart_cr, last_month_cr, last_week_cr])
    return match_sto_ids

NUM_PROCESS = 10
size = math.ceil(len(sto_ids) / NUM_PROCESS)
sub_seqs = []
for proc_idx in range(NUM_PROCESS):
    start_idx = size * proc_idx
    stop_idx = size * (proc_idx + 1)
    sub_seq = sto_ids[start_idx:stop_idx]
    sub_seqs.append(sub_seq)

with Pool(NUM_PROCESS) as pool:
    rlts = pool.map(find_targets, sub_seqs)
    match_sto_infos = []
    for rlt in rlts:
        for match_sto_info in rlt:
            match_sto_infos.append(match_sto_info)
    print(len(match_sto_infos))
    print(match_sto_infos)


In [ ]:
print(match_sto_infos)

In [ ]:
delta = 15
daily_count = int(240/delta)
pic_path = './pic/'
for match_sto_info in match_sto_infos:
#     try:p
    print(match_sto_info)

    sto_id = match_sto_info[0]
    df = BaosAgent.read_freq_df(delta, sto_id)
    df = add_kds_indicators(df)
    df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
    df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)
    df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
    df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)
    df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

    df = df.dropna()
    df = df.iloc[-daily_count * 5 * 36:]
    df = df.reset_index(drop=True)

    cond_long_9t = compute_cond_long(df.k_9t, df.d_9t)
    cond_long_8t = compute_cond_long(df.k_8t, df.d_8t)
    cond_long_7t = compute_cond_long(df.k_7t, df.d_7t)
    cond_long_6t = compute_cond_long(df.k_6t, df.d_6t)
    cond_long_5t = compute_cond_long(df.k_5t, df.d_5t)
    cond_long_4t = compute_cond_long(df.k_4t, df.d_4t)
    cond_long_3t = compute_cond_long(df.k_3t, df.d_3t)
    cond_long_2t = compute_cond_long(df.k_2t, df.d_2t)
    cond_long_1t = compute_cond_long(df.k_1t, df.d_1t)
    cond_all_long = cond_long_1t & cond_long_2t & cond_long_3t & cond_long_4t & cond_long_5t & cond_long_6t & cond_long_7t & cond_long_8t & cond_long_9t
    
    cond_short_9t = compute_cond_short(df.k_9t, df.d_9t)
    cond_short_8t = compute_cond_short(df.k_8t, df.d_8t)
    cond_short_7t = compute_cond_short(df.k_7t, df.d_7t)
    cond_short_6t = compute_cond_short(df.k_6t, df.d_6t)
    cond_short_5t = compute_cond_short(df.k_5t, df.d_5t)
    cond_short_4t = compute_cond_short(df.k_4t, df.d_4t)
    cond_short_3t = compute_cond_short(df.k_3t, df.d_3t)
    cond_short_2t = compute_cond_short(df.k_2t, df.d_2t)
    cond_short_1t = compute_cond_short(df.k_1t, df.d_1t)
    
    cond_high_saturation_9t = compute_high_saturation(df.k_9t)
    cond_high_saturation_8t = compute_high_saturation(df.k_8t)
    cond_high_saturation_7t = compute_high_saturation(df.k_7t)
    cond_high_saturation_6t = compute_high_saturation(df.k_6t)
    cond_high_saturation_5t = compute_high_saturation(df.k_5t)
    cond_high_saturation_4t = compute_high_saturation(df.k_4t)
    cond_high_saturation_3t = compute_high_saturation(df.k_3t)
    cond_high_saturation_2t = compute_high_saturation(df.k_2t)
    cond_high_saturation_1t = compute_high_saturation(df.k_1t)
    
    cond_low_saturation_9t = compute_low_saturation(df.k_9t)
    cond_low_saturation_8t = compute_low_saturation(df.k_8t)
    cond_low_saturation_7t = compute_low_saturation(df.k_7t)
    cond_low_saturation_6t = compute_low_saturation(df.k_6t)
    cond_low_saturation_5t = compute_low_saturation(df.k_5t)
    cond_low_saturation_4t = compute_low_saturation(df.k_4t)
    cond_low_saturation_3t = compute_low_saturation(df.k_3t)
    cond_low_saturation_2t = compute_low_saturation(df.k_2t)
    cond_low_saturation_1t = compute_low_saturation(df.k_1t)

    num_subplots, idx_subplot = 11, 0
    fig = plt.figure(figsize=(.2 * len(df)/daily_count, num_subplots * 1.75))
    fig.patch.set_facecolor('#2A2A2A')

    idx_subplot += 1
    add_ma_subplot(num_subplots, idx_subplot, df)
    idx_subplot += 1
    add_bband_subplot(num_subplots, idx_subplot, df)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_1t, df.d_1t, cond_high_saturation_1t, cond_low_saturation_1t, cond_long_1t, cond_short_1t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_2t, df.d_2t, cond_high_saturation_2t, cond_low_saturation_2t, cond_long_2t, cond_short_2t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_3t, df.d_3t, cond_high_saturation_3t, cond_low_saturation_3t, cond_long_3t, cond_short_3t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_4t, df.d_4t, cond_high_saturation_4t, cond_low_saturation_4t, cond_long_4t, cond_short_4t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_5t, df.d_5t, cond_high_saturation_5t, cond_low_saturation_5t, cond_long_5t, cond_short_5t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_6t, df.d_6t, cond_high_saturation_6t, cond_low_saturation_6t, cond_long_6t, cond_short_6t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_7t, df.d_7t, cond_high_saturation_7t, cond_low_saturation_7t, cond_long_7t, cond_short_7t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_8t, df.d_8t, cond_high_saturation_8t, cond_low_saturation_8t, cond_long_8t, cond_short_8t, df.close, cond_all_long)
    idx_subplot += 1
    add_kd_subplot(num_subplots, idx_subplot, df.k_9t, df.d_9t, cond_high_saturation_9t, cond_low_saturation_9t, cond_long_9t, cond_short_9t, df.close, cond_all_long)

    curp = df.iloc[-1].close
    last_week_minp = min(df.close.iloc[-daily_count * 5:])
    last_week_cr = (curp - last_week_minp) / last_week_minp
    last_month_minp = min(df.close.iloc[-daily_count * 20:])
    last_month_cr = (curp - last_month_minp) / last_month_minp
    last_quart_minp = min(df.close.iloc[-daily_count * 60:])
    last_quart_cr = (curp - last_quart_minp) / last_quart_minp
    last_hyear_minp = min(df.close.iloc[-daily_count * 120:])
    last_hyear_cr = (curp - last_hyear_minp) / last_hyear_minp
    
    plt.show()
    
#     fn_fig = '{}_{:>6.2%}_{:>6.2%}_{:>6.2%}_{:>6.2%}.png'.format(sto_id, last_hyear_cr, last_quart_cr, last_month_cr, last_week_cr)
#     print(fn_fig)
#     plt.savefig(pic_path+fn_fig, dpi=256, bbox_inches='tight', facecolor='#2A2A2A', edgecolor='none')
    plt.close('all')
    print()
#     clear_output()
#     except Exception as e:
#         print(sto_id, e)

In [ ]:
########################

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()
jq.get_query_count()

### Save target figures

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()

delta = 15
daily_count = int(240/delta)

target_path = './target/'
target_fns = os.listdir(target_path)
target_fns.sort()
for target_fn in target_fns:
    
    if not target_fn.startswith('matches'):
        continue
    if os.path.isdir(target_path + target_fn):
        continue
    
    pic_path = target_path + target_fn[:-4] + '/'
    ts_jq = datetime.strptime(target_fn[8:-4], '%Y-%m-%d-%H%M')
    ts_jq = datetime.strftime(ts_jq, '%Y-%m-%d %H:%M:%S')
    if not os.path.exists(pic_path):
        os.mkdir(pic_path)
        target_df = pd.read_csv(target_path + target_fn, dtype={'sto_id':str})
        for idx in target_df.index:
            try:
                record = target_df.iloc[idx]
                sto_id = record.sto_id
                df = BaosAgent.read_freq_df(delta, sto_id)
                print('get realtime data from JQData ...')
                df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
                print('get realtime data from JQData accomplished.')
                df = add_kds_indicators(df)
                df['CLOSE_MA_18d'] = talib.MA(df.close,  timeperiod=daily_count * 18)
                df['CLOSE_MA_50d'] = talib.MA(df.close,  timeperiod=daily_count * 50)
                df['VOL_MA_18d']   = talib.MA(df.volume, timeperiod=daily_count * 18)
                df['VOL_MA_50d']   = talib.MA(df.volume, timeperiod=daily_count * 50)
                df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

                df = df.dropna()
                df = df.iloc[-daily_count * 5 * 12:]
                df = df.reset_index(drop=True)

                cond_long_9t = compute_cond_long(df.k_9t, df.d_9t)
                cond_long_8t = compute_cond_long(df.k_8t, df.d_8t)
                cond_long_7t = compute_cond_long(df.k_7t, df.d_7t)
                cond_long_6t = compute_cond_long(df.k_6t, df.d_6t)
                cond_long_5t = compute_cond_long(df.k_5t, df.d_5t)
                cond_long_4t = compute_cond_long(df.k_4t, df.d_4t)
                cond_long_3t = compute_cond_long(df.k_3t, df.d_3t)
                cond_long_2t = compute_cond_long(df.k_2t, df.d_2t)
                cond_long_1t = compute_cond_long(df.k_1t, df.d_1t)
                cond_all_long = cond_long_1t & cond_long_2t & cond_long_3t & cond_long_4t & cond_long_5t & cond_long_6t & cond_long_7t & cond_long_8t & cond_long_9t

                cond_short_9t = compute_cond_short(df.k_9t, df.d_9t)
                cond_short_8t = compute_cond_short(df.k_8t, df.d_8t)
                cond_short_7t = compute_cond_short(df.k_7t, df.d_7t)
                cond_short_6t = compute_cond_short(df.k_6t, df.d_6t)
                cond_short_5t = compute_cond_short(df.k_5t, df.d_5t)
                cond_short_4t = compute_cond_short(df.k_4t, df.d_4t)
                cond_short_3t = compute_cond_short(df.k_3t, df.d_3t)
                cond_short_2t = compute_cond_short(df.k_2t, df.d_2t)
                cond_short_1t = compute_cond_short(df.k_1t, df.d_1t)

                cond_high_saturation_9t = compute_high_saturation(df.k_9t)
                cond_high_saturation_8t = compute_high_saturation(df.k_8t)
                cond_high_saturation_7t = compute_high_saturation(df.k_7t)
                cond_high_saturation_6t = compute_high_saturation(df.k_6t)
                cond_high_saturation_5t = compute_high_saturation(df.k_5t)
                cond_high_saturation_4t = compute_high_saturation(df.k_4t)
                cond_high_saturation_3t = compute_high_saturation(df.k_3t)
                cond_high_saturation_2t = compute_high_saturation(df.k_2t)
                cond_high_saturation_1t = compute_high_saturation(df.k_1t)

                cond_low_saturation_9t = compute_low_saturation(df.k_9t)
                cond_low_saturation_8t = compute_low_saturation(df.k_8t)
                cond_low_saturation_7t = compute_low_saturation(df.k_7t)
                cond_low_saturation_6t = compute_low_saturation(df.k_6t)
                cond_low_saturation_5t = compute_low_saturation(df.k_5t)
                cond_low_saturation_4t = compute_low_saturation(df.k_4t)
                cond_low_saturation_3t = compute_low_saturation(df.k_3t)
                cond_low_saturation_2t = compute_low_saturation(df.k_2t)
                cond_low_saturation_1t = compute_low_saturation(df.k_1t)
                
                num_subplots, idx_subplot = 11, 0
                fig = plt.figure(figsize=(.6 * len(df)/daily_count, num_subplots * 1.75))
                fig.patch.set_facecolor('#2A2A2A')

                idx_subplot += 1
                add_ma_subplot(num_subplots, idx_subplot, df)
                idx_subplot += 1
                add_bband_subplot(num_subplots, idx_subplot, df)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_1t, df.d_1t, cond_high_saturation_1t, cond_low_saturation_1t, cond_long_1t, cond_short_1t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_2t, df.d_2t, cond_high_saturation_2t, cond_low_saturation_2t, cond_long_2t, cond_short_2t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_3t, df.d_3t, cond_high_saturation_3t, cond_low_saturation_3t, cond_long_3t, cond_short_3t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_4t, df.d_4t, cond_high_saturation_4t, cond_low_saturation_4t, cond_long_4t, cond_short_4t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_5t, df.d_5t, cond_high_saturation_5t, cond_low_saturation_5t, cond_long_5t, cond_short_5t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_6t, df.d_6t, cond_high_saturation_6t, cond_low_saturation_6t, cond_long_6t, cond_short_6t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_7t, df.d_7t, cond_high_saturation_7t, cond_low_saturation_7t, cond_long_7t, cond_short_7t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_8t, df.d_8t, cond_high_saturation_8t, cond_low_saturation_8t, cond_long_8t, cond_short_8t, df.close, cond_all_long)
                idx_subplot += 1
                add_kd_subplot(num_subplots, idx_subplot, df.k_9t, df.d_9t, cond_high_saturation_9t, cond_low_saturation_9t, cond_long_9t, cond_short_9t, df.close, cond_all_long)

                curp = df.iloc[-1].close
                last_week_minp = min(df.close.iloc[-daily_count * 5:])
                last_week_cr = (curp - last_week_minp) / last_week_minp
                last_month_minp = min(df.close.iloc[-daily_count * 20:])
                last_month_cr = (curp - last_month_minp) / last_month_minp
                last_quart_minp = min(df.close.iloc[-daily_count * 60:])
                last_quart_cr = (curp - last_quart_minp) / last_quart_minp
                last_hyear_minp = min(df.close.iloc[-daily_count * 120:])
                last_hyear_cr = (curp - last_hyear_minp) / last_hyear_minp
                fn_fig = '{}_{:>5.2%}_{:>5.2%}_{:>5.2%}_{:>5.2%}.png'.format(sto_id, record.last_hyear_cr, record.last_quart_cr, record.last_month_cr, record.last_week_cr)
                print(fn_fig)
                plt.savefig(pic_path+fn_fig, dpi=256, bbox_inches='tight', facecolor='#2A2A2A', edgecolor='none')
                plt.close('all')
#                 clear_output()
            except Exception as e:
                print(sto_id, e)

jq.logout()

In [ ]:
jq.auth('15808061188', 'allan2jq')
clear_output()
# jq.get_query_count()

delta = 15
daily_count = int(240/delta)

###### read sto_ids from file
# sto_ids = get_freq_file_sto_ids(delta)
# sto_id = random.sample(match_sto_ids, 1)[0]

###### custome sto_id
# sto_id = '002030'

###### read sto_ids from MATCHE file
matches_df = pd.read_csv('./target/matches_2020-02-09-1500.csv', dtype={'sto_id':str})
# sto_ids_backup = matches_df.sto_id.tolist()
# sto_id = random.sample(sto_ids_backup, 1)[0]



for idx in matches_df.index:
    try:
        record = matches_df.iloc[idx]
        sto_id = record.sto_id
        df = BaosAgent.read_freq_df(delta, sto_id)
#         print('get data from baostock ...')
#         df = BaosAgent.get_freq_data_from_bs(sto_id, '2014-01-01', delta)
#         print('get data from baostock accomplished.')
        # df.time = df.time.str[:-5]
        # df.code = df.code.str[3:]
        # df[['open', 'high', 'low', 'close', 'volume']] = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
        # df.open = np.round(df.open, 2)
        # df.high = np.round(df.high, 2)
        # df.low = np.round(df.low, 2)
        # df.close = np.round(df.close, 2)

        print('get realtime data from JQData ...')
        df = DataUtil.add_rt_data(sto_id, df, delta, JQDataUtil.get_end_ts(delta))
        print('get realtime data from JQData accomplished.')

        org_size = len(df)
        print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])


        daily_count = int(240/delta)
        df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
        df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)
        df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
        df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)
        df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

        factor_qut = daily_count * 60
        factor_mon = daily_count * 20
        factor_wek = daily_count * 5
        factor_day = daily_count * 1

        df['k_qut'], df['d_qut'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_qut, slowk_period=3*factor_qut, slowd_period=3*factor_qut)
        df['k_mon'], df['d_mon'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_mon, slowk_period=3*factor_mon, slowd_period=3*factor_mon)
        df['k_wek'], df['d_wek'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_wek, slowk_period=3*factor_wek, slowd_period=3*factor_wek)
        df['k_day'], df['d_day'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_day, slowk_period=3*factor_day, slowd_period=3*factor_day)
        df['k_15m'], df['d_15m'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9,            slowk_period=3,            slowd_period=3)

        df = df.dropna()
        print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])

        #     if len(df) >= daily_count * 5 * 20:

        print(org_size - len(df))
        df = df.iloc[-daily_count * 5 * 50:]
        df = df.reset_index(drop=True)
        print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])
        long, short = compute_longshort_index(df.iloc[-1])
        print('{:>2}:{:<2}'.format(long, short))

        cond_qut_long  = (df.k_qut > df.d_qut) & (df.k_qut > 20) | (df.k_qut > 80)
        cond_qut_short = (df.k_qut < df.d_qut) & (df.k_qut < 80) | (df.k_qut < 20)
        #     cond_qut_long  = (df.k_qut > df.k_qut.shift(1)) & (df.d_qut > df.d_qut.shift(1))
        #     cond_qut_short = (df.k_qut < df.k_qut.shift(1)) & (df.d_qut < df.d_qut.shift(1))
        cond_qut_consolidation = ~(cond_qut_long | cond_qut_short)
        cond_qut_high_saturation = (df.k_qut > 80).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
        cond_qut_low_saturation  = (df.k_qut < 20).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1

        cond_mon_long  = (df.k_mon > df.d_mon) & (df.k_mon > 20) | (df.k_mon > 80)
        cond_mon_short = (df.k_mon < df.d_mon) & (df.k_mon < 80) | (df.k_mon < 20)
        #     cond_mon_long  = (df.k_mon > df.k_mon.shift(1)) & (df.d_mon > df.d_mon.shift(1))
        #     cond_mon_short = (df.k_mon < df.k_mon.shift(1)) & (df.d_mon < df.d_mon.shift(1))
        cond_mon_consolidation = ~(cond_mon_long | cond_mon_short)
        cond_mon_high_saturation = (df.k_mon > 80).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
        cond_mon_low_saturation  = (df.k_mon < 20).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1

        cond_wek_long  = (df.k_wek > df.d_wek) & (df.k_wek > 20) | (df.k_wek > 80)
        cond_wek_short = (df.k_wek < df.d_wek) & (df.k_wek < 80) | (df.k_wek < 20)
        #     cond_wek_long  = ((df.k_wek > df.k_wek.shift(1)) & (df.d_wek > df.d_wek.shift(1))) | (df.k_wek > 80)
        #     cond_wek_short = (df.k_wek < df.k_wek.shift(1)) & (df.d_wek < df.d_wek.shift(1)) & (df.k_wek < 80)
        cond_wek_consolidation = ~(cond_wek_long | cond_wek_short)
        cond_wek_high_saturation = (df.k_wek > 80).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
        cond_wek_low_saturation  = (df.k_wek < 20).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1

        cond_day_long  = (df.k_day > df.d_day) & (df.k_day > 20) | (df.k_day > 80)
        cond_day_short = (df.k_day < df.d_day) & (df.k_day < 80) | (df.k_day < 20)
        cond_day_high_saturation = (df.k_day > 80).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1
        cond_day_low_saturation  = (df.k_day < 20).rolling(daily_count * 3).apply(lambda x:x.all(), raw=True) == 1

        num_subplots, idx_subplot = 6, 0
        fig = plt.figure(figsize=(.17 * len(df)/daily_count, num_subplots * 3))
        fig.patch.set_facecolor('#2A2A2A')

        idx_subplot += 1
        add_ma_subplot(num_subplots, idx_subplot, df)

        idx_subplot += 1
        add_bband_subplot(num_subplots, idx_subplot, df)

        idx_subplot += 1
        add_kd_day_subplot(num_subplots, idx_subplot, df)

        idx_subplot += 1
        add_kd_wek_subplot(num_subplots, idx_subplot, df)

        idx_subplot += 1
        add_kd_mon_subplot(num_subplots, idx_subplot, df)

        idx_subplot += 1
        add_kd_qut_subplot(num_subplots, idx_subplot, df)

        fn_fig = '{}_{:>6.2%}_{:>6.2%}_{:>6.2%}_{:>6.2%}.png'.format(sto_id, record.last_hyear_cr, record.last_quart_cr, record.last_month_cr, record.last_week_cr)
        print(fn_fig)
        plt.savefig('./pic/'+fn_fig, dpi=256, bbox_inches='tight', facecolor='#2A2A2A', edgecolor='none')
        plt.close('all')
    except Exception as e:
        print(sto_id, e)
    finally:
        clear_output()


jq.logout()


### MA Long Short Evaluation

In [ ]:
delta = 15
daily_count = int(240/delta)
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]
df = BaosAgent.read_freq_df(delta, sto_id)

df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)

df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)

df = df.dropna()

print(len(df))
# df = df.iloc[-daily_count * 5 * 50:]
df = df.reset_index(drop=True)

tgt_idxes = []
# cond = (df.k_day.shift(1) < df.d_day.shift(1)) & (df.d_day < df.k_day)
# cond = (df.k_day < 20) & cond
# tgt_idxes = df[cond].index

fig = plt.figure(figsize=(.25 * len(df)/daily_count, 5))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)
        
ax = plt.subplot(1,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.CLOSE_MA_18d, lw=.5, color='cyan')
ax.plot(xaxis_idxes, df.CLOSE_MA_50d, lw=.5, color='lime')
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')

cond_long  = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
cond_short = (df.CLOSE_MA_18d < df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d < df.CLOSE_MA_50d.shift(1))
cond_consolidation = ((df.CLOSE_MA_18d <= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d >= df.CLOSE_MA_50d.shift(1))) | \
                     ((df.CLOSE_MA_18d >= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d <= df.CLOSE_MA_50d.shift(1)))

# cond_long  = (df.VOL_MA_18d > df.VOL_MA_18d.shift(1)) & (df.VOL_MA_50d > df.VOL_MA_50d.shift(1))
# cond_short = (df.VOL_MA_18d < df.VOL_MA_18d.shift(1)) & (df.VOL_MA_50d < df.VOL_MA_50d.shift(1))
# cond_consolidation = ((df.VOL_MA_18d <= df.VOL_MA_18d.shift(1)) & (df.VOL_MA_50d >= df.VOL_MA_50d.shift(1))) | \
#                      ((df.VOL_MA_18d >= df.VOL_MA_18d.shift(1)) & (df.VOL_MA_50d <= df.VOL_MA_50d.shift(1)))

ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_long,  facecolor='red',  alpha=0.3)
ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_short, facecolor='limegreen', alpha=0.3)
ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_consolidation, facecolor='orange', alpha=0.3)
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')


plt.show()

### BBAND Long Short Evaluation

In [ ]:
delta = 15
daily_count = int(240/delta)
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]
df = BaosAgent.read_freq_df(delta, sto_id)


### init bband features
df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

df = df.dropna()
print(len(df))
# df = df.iloc[-daily_count * 5 * 50:]
df = df.reset_index(drop=True)


cond_long  = (df.close > df.BBAND_upper)


tgt_idxes = []
# cond = (df.k_day.shift(1) < df.d_day.shift(1)) & (df.d_day < df.k_day)
# cond = (df.k_day < 20) & cond
# tgt_idxes = df[cond].index

fig = plt.figure(figsize=(.25 * len(df)/daily_count, 5))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)

ax = plt.subplot(1,1,1)
PlotTool.custom_ax(ax)
ax.plot(xaxis_idxes, df.BBAND_upper, lw=.3, color='red')
ax.plot(xaxis_idxes, df.BBAND_lower, lw=.3, color='lime')
ax.plot(xaxis_idxes, df.close, lw=.5, color='orange')
ax.fill_between(xaxis_idxes, min(df.close), max(df.close), where=cond_long,  facecolor='red',  alpha=0.2)
plt.sca(ax)
plt.xticks(xaxis_idxes[x_axis_slice], fontsize=8, rotation=10, color='w')
plt.yticks(fontsize=8, color='w')


plt.show()


### Combination Evaluation

In [ ]:
### init raw data
delta = 15
daily_count = int(240/delta)
sto_ids = get_freq_file_sto_ids(delta)
sto_id = random.sample(sto_ids, 1)[0]
df = BaosAgent.read_freq_df(delta, sto_id)


### init ta features
df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)
df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)

df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

df = df.dropna()
# df = df.iloc[-daily_count * 5 * 50:]
df = df.reset_index(drop=True)


### inin evaluation conditions
cond_ma_long  = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
cond_ma_short = (df.CLOSE_MA_18d < df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d < df.CLOSE_MA_50d.shift(1))
cond_ma_consolidation = ((df.CLOSE_MA_18d <= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d >= df.CLOSE_MA_50d.shift(1))) | \
                     ((df.CLOSE_MA_18d >= df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d <= df.CLOSE_MA_50d.shift(1)))
cond_bband_long  = (df.close > df.BBAND_upper)
cond_bband_short = (df.close < df.BBAND_lower)


tgt_idxes = []
# cond = (df.k_day.shift(1) < df.d_day.shift(1)) & (df.d_day < df.k_day)
# cond = (df.k_day < 20) & cond
# tgt_idxes = df[cond].index


### plot figure
num_subplots, idx_subplot = 2, 0
fig = plt.figure(figsize=(.25 * len(df)/daily_count, num_subplots * 3))
fig.patch.set_facecolor('#2A2A2A')
xaxis_idxes = df.index.values
x_axis_slice = slice(0, -1, daily_count * 5)

idx_subplot += 1
add_ma_subplot(num_subplots, idx_subplot, df)

idx_subplot += 1
add_bband_subplot(num_subplots, idx_subplot, df)

plt.show()

In [ ]:
df[df.close == df.close.rolling(daily_count * 120).max()]

# df.close.rolling(daily_count * 120).max()

In [ ]:
delta = 15
daily_count = int(240/delta)
sto_ids = get_freq_file_sto_ids(delta)
match_sto_ids = []
for sto_id in tqdm(sto_ids):
    df = BaosAgent.read_freq_df(delta, sto_id)
    org_size = len(df)
    if org_size > 14397 + daily_count * 5 * 20:
#         print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])
        df['CLOSE_MA_18d'] = talib.MA(df.close, timeperiod=daily_count * 18)
        df['CLOSE_MA_50d'] = talib.MA(df.close, timeperiod=daily_count * 50)
#         df['VOL_MA_18d'] = talib.MA(df.volume, timeperiod=daily_count * 18)
#         df['VOL_MA_50d'] = talib.MA(df.volume, timeperiod=daily_count * 50)
#         df['BBAND_upper'], df['BBAND_middle'], df['BBAND_lower'] = talib.BBANDS(df.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

        factor_qut = daily_count * 60
        factor_mon = daily_count * 20
        factor_wek = daily_count * 5
        factor_day = daily_count * 1

        df['k_qut'], df['d_qut'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_qut, slowk_period=3*factor_qut, slowd_period=3*factor_qut)
        df['k_mon'], df['d_mon'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_mon, slowk_period=3*factor_mon, slowd_period=3*factor_mon)
        df['k_wek'], df['d_wek'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_wek, slowk_period=3*factor_wek, slowd_period=3*factor_wek)
        df['k_day'], df['d_day'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9*factor_day, slowk_period=3*factor_day, slowd_period=3*factor_day)
#         df['k_15m'], df['d_15m'] = talib.STOCH(df.high, df.low, df.close, fastk_period=9,                slowk_period=3,                slowd_period=3)

        df = df.dropna()
#         print(org_size - len(df))
        # df = df.iloc[-daily_count * 5 * 50:]
        df = df.reset_index(drop=True)
#         print(sto_id, len(df), df.time.iloc[0], df.time.iloc[-1])
        cond_ma_long = (df.CLOSE_MA_18d > df.CLOSE_MA_18d.shift(1)) & (df.CLOSE_MA_50d > df.CLOSE_MA_50d.shift(1))
        long, short = compute_longshort_index(df.iloc[-1])
        if long == 10 and short == 0 and cond_ma_long.iloc[-1]:
            print('{} {:>2}:{:<2}'.format(sto_id, long, short))
            match_sto_ids.append(sto_id)
print(len(match_sto_ids))

In [ ]:
find_matches(sto_ids[:10])

In [ ]:
df[['k_day', 'k_wek', 'k_mon', 'k_qut']].iloc[-1]

In [ ]:
daily_count = 1
df_day['CLOSE_MA_18d'] = talib.MA(df_day.close, timeperiod=daily_count * 18)
df_day['CLOSE_MA_50d'] = talib.MA(df_day.close, timeperiod=daily_count * 50)

df_day['VOL_MA_18d'] = talib.MA(df_day.volume, timeperiod=daily_count * 18)
df_day['VOL_MA_50d'] = talib.MA(df_day.volume, timeperiod=daily_count * 50)

df_day['BBAND_upper'], df_day['BBAND_middle'], df_day['BBAND_lower'] = talib.BBANDS(df_day.close, timeperiod=daily_count * 5, nbdevup=1, nbdevdn=1, matype=0)

# fast, slow = 14, 3
# df_day['k_qut'], df_day['d_qut'] = compute_kd(df_day, fast * 60, slow * 60)
# df_day['k_mon'], df_day['d_mon'] = compute_kd(df_day, fast * 20, slow * 20)
# df_day['k_wek'], df_day['d_wek'] = compute_kd(df_day, fast * 5,  slow * 5)
# df_day['k_day'], df_day['d_day'] = compute_kd(df_day, fast,      slow)

df_day['k_qut'], df_day['d_qut'] = talib.STOCH(df_day.high, df_day.low, df_day.close, fastk_period=9*daily_count*60, slowk_period=3*daily_count*60, slowd_period=3*daily_count*60)
df_day['k_mon'], df_day['d_mon'] = talib.STOCH(df_day.high, df_day.low, df_day.close, fastk_period=9*daily_count*20, slowk_period=3*daily_count*20, slowd_period=3*daily_count*20)
df_day['k_wek'], df_day['d_wek'] = talib.STOCH(df_day.high, df_day.low, df_day.close, fastk_period=9*daily_count*5,  slowk_period=3*daily_count*5,  slowd_period=3*daily_count*5)
df_day['k_day'], df_day['d_day'] = talib.STOCH(df_day.high, df_day.low, df_day.close, fastk_period=9*daily_count,    slowk_period=3*daily_count,    slowd_period=3*daily_count)
df_day['k_15m'], df_day['d_15m'] = talib.STOCH(df_day.high, df_day.low, df_day.close, fastk_period=9,                slowk_period=3,                slowd_period=3)

df_day = df_day.dropna()
df_day = df_day.reset_index(drop=True)
print(df_day.time.iloc[0][:8], df_day.time.iloc[0][8:12])

long, short = compute_longshort_index(df_day.iloc[-1])
print('{:>2}:{:<2}'.format(long, short))

num_subplots, idx_subplot = 6, 0
fig = plt.figure(figsize=(.15 * len(df_day)/daily_count, num_subplots * 3))
fig.patch.set_facecolor('#2A2A2A')

idx_subplot += 1
add_ma_subplot(num_subplots, idx_subplot, df_day)

idx_subplot += 1
add_bband_subplot(num_subplots, idx_subplot, df_day)

idx_subplot += 1
add_kd_day_subplot(num_subplots, idx_subplot, df_day)

idx_subplot += 1
add_kd_wek_subplot(num_subplots, idx_subplot, df_day)

idx_subplot += 1
add_kd_mon_subplot(num_subplots, idx_subplot, df_day)

idx_subplot += 1
add_kd_qut_subplot(num_subplots, idx_subplot, df_day)

plt.show()
plt.close('all')